In [1]:
!pip install optuna grande entmax

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorf

In [2]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import accuracy_score, matthews_corrcoef, make_scorer
from scipy.stats import shapiro
import optuna
from GRANDE import GRANDE
from ncart import NCARClassifier

In [3]:
# This is a function that loads training and testing data for 10-fold cross validation
def prepare_train_test_dataset(datasets, test_idx):
  test_df = datasets[test_idx]
  train_df = pd.concat([datasets[i] for i in range(len(datasets)) if i != test_idx])

  X_train = train_df.iloc[:, 1:-1].values
  X_test = test_df.iloc[:, 1:-1].values
  y_train = train_df.iloc[:, -1].values
  y_test = test_df.iloc[:, -1].values

  return X_train, X_test, y_train, y_test

In [4]:
import torch

def train_with_base_ncart(datasets, key):
  print("Currently training NCART model with dataset key:", key)

  # Create a Pandas DataFrame to store all experiment results
  results_df = pd.DataFrame(columns=['Fold', 'Accuracy', 'MCC'])

  # Perform 10-fold cross validation
  dataset_list = datasets[key]

  for i in range(10):
    X_train, X_test, y_train, y_test = prepare_train_test_dataset(dataset_list, i)

    # Train the model
    model = NCARClassifier()
    model.fit(X=X_train, y=y_train, X_val=X_test, y_val=y_test)

    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)

    print("Fold", i+1, "Accuracy:", accuracy, "MCC:", mcc)
    results_df.loc[i] = [i+1, accuracy, mcc]

  return results_df

In [5]:
def create_objective_for_ncart(X_train, X_test, y_train, y_test):
  def objective(trial):
      # Suggest hyperparameters for GRANDE
      params = {
          "n_trees": trial.suggest_int("n_trees", 8, 128, step=8),
          "n_selected": trial.suggest_int("n_selected", 4, 32, step=4),
          "n_layers": trial.suggest_int("n_layers", 1, 8),
          "mask_type": trial.suggest_categorical("mask_type", ["sparsemax", "entmax"]),
      }

      base_model = NCARClassifier(
          n_trees=params["n_trees"],
          n_selected=params["n_selected"],
          n_layers=params["n_layers"],
          mask_type=params["mask_type"],
      )
      base_model.fit(X=X_train, y=y_train, X_val=X_test, y_val=y_test)

      y_pred = base_model.predict(X_test)
      mcc = matthews_corrcoef(y_test, y_pred)

      # Return the accuracy
      return mcc

  return objective

In [6]:
def train_with_best_hyperparameters(datasets, key):
  print("Currently training NCART model with dataset key:", key)

  # Create a Pandas DataFrame to store all experiment results
  results_df = pd.DataFrame(columns=['Fold', 'Accuracy', 'MCC'])

  # Perform 10-fold cross validation
  dataset_list = datasets[key]

  # Use the first data as the test for the HPO
  X_train, X_test, y_train, y_test = prepare_train_test_dataset(dataset_list, 0)

  objective_function = create_objective_for_ncart(X_train, X_test, y_train, y_test)

  study = optuna.create_study(direction='maximize')
  study.optimize(objective_function, n_trials=15, gc_after_trial=True)

  best_params = study.best_params
  print("Best hyperparameters:", best_params)

  # Use best hyperparams to conduct a 10-fold cross validation
  for i in range(10):
    X_train, X_test, y_train, y_test = prepare_train_test_dataset(dataset_list, i)

    # Train the model
    model = NCARClassifier(
        n_trees=best_params["n_trees"],
        n_selected=best_params["n_selected"],
        n_layers=best_params["n_layers"],
        mask_type=best_params["mask_type"],
    )
    model.fit(X=X_train, y=y_train, X_val=X_test, y_val=y_test)

    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)

    print("Fold", i+1, "Accuracy:", accuracy, "MCC:", mcc)
    results_df.loc[i] = [i+1, accuracy, mcc]

  return results_df

In [7]:
# Load the datasets

datasets = {}

# All

II_all_nopcc_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/nopcc/fold_1.csv')
II_all_nopcc_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/nopcc/fold_2.csv')
II_all_nopcc_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/nopcc/fold_3.csv')
II_all_nopcc_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/nopcc/fold_4.csv')
II_all_nopcc_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/nopcc/fold_5.csv')
II_all_nopcc_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/nopcc/fold_6.csv')
II_all_nopcc_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/nopcc/fold_7.csv')
II_all_nopcc_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/nopcc/fold_8.csv')
II_all_nopcc_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/nopcc/fold_9.csv')
II_all_nopcc_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/nopcc/fold_10.csv')
datasets['II_all_nopcc'] = [II_all_nopcc_1, II_all_nopcc_2, II_all_nopcc_3, II_all_nopcc_4, II_all_nopcc_5,
                                       II_all_nopcc_6, II_all_nopcc_7, II_all_nopcc_8, II_all_nopcc_9, II_all_nopcc_10]

II_all_pcc95_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc95/fold_1.csv')
II_all_pcc95_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc95/fold_2.csv')
II_all_pcc95_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc95/fold_3.csv')
II_all_pcc95_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc95/fold_4.csv')
II_all_pcc95_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc95/fold_5.csv')
II_all_pcc95_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc95/fold_6.csv')
II_all_pcc95_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc95/fold_7.csv')
II_all_pcc95_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc95/fold_8.csv')
II_all_pcc95_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc95/fold_9.csv')
II_all_pcc95_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc95/fold_10.csv')
datasets['II_all_pcc95'] = [II_all_pcc95_1, II_all_pcc95_2, II_all_pcc95_3, II_all_pcc95_4, II_all_pcc95_5,
                                       II_all_pcc95_6, II_all_pcc95_7, II_all_pcc95_8, II_all_pcc95_9, II_all_pcc95_10]

II_all_pcc75_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc75/fold_1.csv')
II_all_pcc75_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc75/fold_2.csv')
II_all_pcc75_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc75/fold_3.csv')
II_all_pcc75_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc75/fold_4.csv')
II_all_pcc75_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc75/fold_5.csv')
II_all_pcc75_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc75/fold_6.csv')
II_all_pcc75_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc75/fold_7.csv')
II_all_pcc75_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc75/fold_8.csv')
II_all_pcc75_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc75/fold_9.csv')
II_all_pcc75_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/all/cv/pcc75/fold_10.csv')
datasets['II_all_pcc75'] = [II_all_pcc75_1, II_all_pcc75_2, II_all_pcc75_3, II_all_pcc75_4, II_all_pcc75_5,
                                       II_all_pcc75_6, II_all_pcc75_7, II_all_pcc75_8, II_all_pcc75_9, II_all_pcc75_10]

# 550

II_550_nopcc_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/nopcc/fold_1.csv')
II_550_nopcc_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/nopcc/fold_2.csv')
II_550_nopcc_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/nopcc/fold_3.csv')
II_550_nopcc_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/nopcc/fold_4.csv')
II_550_nopcc_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/nopcc/fold_5.csv')
II_550_nopcc_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/nopcc/fold_6.csv')
II_550_nopcc_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/nopcc/fold_7.csv')
II_550_nopcc_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/nopcc/fold_8.csv')
II_550_nopcc_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/nopcc/fold_9.csv')
II_550_nopcc_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/nopcc/fold_10.csv')
datasets['II_550_nopcc'] = [II_550_nopcc_1, II_550_nopcc_2, II_550_nopcc_3, II_550_nopcc_4, II_550_nopcc_5,
                                       II_550_nopcc_6, II_550_nopcc_7, II_550_nopcc_8, II_550_nopcc_9, II_550_nopcc_10]

II_550_pcc95_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc95/fold_1.csv')
II_550_pcc95_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc95/fold_2.csv')
II_550_pcc95_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc95/fold_3.csv')
II_550_pcc95_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc95/fold_4.csv')
II_550_pcc95_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc95/fold_5.csv')
II_550_pcc95_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc95/fold_6.csv')
II_550_pcc95_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc95/fold_7.csv')
II_550_pcc95_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc95/fold_8.csv')
II_550_pcc95_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc95/fold_9.csv')
II_550_pcc95_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc95/fold_10.csv')
datasets['II_550_pcc95'] = [II_550_pcc95_1, II_550_pcc95_2, II_550_pcc95_3, II_550_pcc95_4, II_550_pcc95_5,
                                       II_550_pcc95_6, II_550_pcc95_7, II_550_pcc95_8, II_550_pcc95_9, II_550_pcc95_10]

II_550_pcc75_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc75/fold_1.csv')
II_550_pcc75_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc75/fold_2.csv')
II_550_pcc75_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc75/fold_3.csv')
II_550_pcc75_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc75/fold_4.csv')
II_550_pcc75_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc75/fold_5.csv')
II_550_pcc75_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc75/fold_6.csv')
II_550_pcc75_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc75/fold_7.csv')
II_550_pcc75_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc75/fold_8.csv')
II_550_pcc75_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc75/fold_9.csv')
II_550_pcc75_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/550/cv/pcc75/fold_10.csv')
datasets['II_550_pcc75'] = [II_550_pcc75_1, II_550_pcc75_2, II_550_pcc75_3, II_550_pcc75_4, II_550_pcc75_5,
                                       II_550_pcc75_6, II_550_pcc75_7, II_550_pcc75_8, II_550_pcc75_9, II_550_pcc75_10]

# 450

II_450_nopcc_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/nopcc/fold_1.csv')
II_450_nopcc_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/nopcc/fold_2.csv')
II_450_nopcc_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/nopcc/fold_3.csv')
II_450_nopcc_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/nopcc/fold_4.csv')
II_450_nopcc_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/nopcc/fold_5.csv')
II_450_nopcc_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/nopcc/fold_6.csv')
II_450_nopcc_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/nopcc/fold_7.csv')
II_450_nopcc_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/nopcc/fold_8.csv')
II_450_nopcc_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/nopcc/fold_9.csv')
II_450_nopcc_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/nopcc/fold_10.csv')
datasets['II_450_nopcc'] = [II_450_nopcc_1, II_450_nopcc_2, II_450_nopcc_3, II_450_nopcc_4, II_450_nopcc_5,
                                       II_450_nopcc_6, II_450_nopcc_7, II_450_nopcc_8, II_450_nopcc_9, II_450_nopcc_10]

II_450_pcc95_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc95/fold_1.csv')
II_450_pcc95_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc95/fold_2.csv')
II_450_pcc95_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc95/fold_3.csv')
II_450_pcc95_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc95/fold_4.csv')
II_450_pcc95_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc95/fold_5.csv')
II_450_pcc95_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc95/fold_6.csv')
II_450_pcc95_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc95/fold_7.csv')
II_450_pcc95_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc95/fold_8.csv')
II_450_pcc95_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc95/fold_9.csv')
II_450_pcc95_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc95/fold_10.csv')
datasets['II_450_pcc95'] = [II_450_pcc95_1, II_450_pcc95_2, II_450_pcc95_3, II_450_pcc95_4, II_450_pcc95_5,
                                       II_450_pcc95_6, II_450_pcc95_7, II_450_pcc95_8, II_450_pcc95_9, II_450_pcc95_10]

II_450_pcc75_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc75/fold_1.csv')
II_450_pcc75_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc75/fold_2.csv')
II_450_pcc75_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc75/fold_3.csv')
II_450_pcc75_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc75/fold_4.csv')
II_450_pcc75_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc75/fold_5.csv')
II_450_pcc75_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc75/fold_6.csv')
II_450_pcc75_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc75/fold_7.csv')
II_450_pcc75_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc75/fold_8.csv')
II_450_pcc75_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc75/fold_9.csv')
II_450_pcc75_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/450/cv/pcc75/fold_10.csv')
datasets['II_450_pcc75'] = [II_450_pcc75_1, II_450_pcc75_2, II_450_pcc75_3, II_450_pcc75_4, II_450_pcc75_5,
                                       II_450_pcc75_6, II_450_pcc75_7, II_450_pcc75_8, II_450_pcc75_9, II_450_pcc75_10]

# 350

II_350_nopcc_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/nopcc/fold_1.csv')
II_350_nopcc_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/nopcc/fold_2.csv')
II_350_nopcc_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/nopcc/fold_3.csv')
II_350_nopcc_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/nopcc/fold_4.csv')
II_350_nopcc_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/nopcc/fold_5.csv')
II_350_nopcc_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/nopcc/fold_6.csv')
II_350_nopcc_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/nopcc/fold_7.csv')
II_350_nopcc_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/nopcc/fold_8.csv')
II_350_nopcc_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/nopcc/fold_9.csv')
II_350_nopcc_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/nopcc/fold_10.csv')
datasets['II_350_nopcc'] = [II_350_nopcc_1, II_350_nopcc_2, II_350_nopcc_3, II_350_nopcc_4, II_350_nopcc_5,
                                       II_350_nopcc_6, II_350_nopcc_7, II_350_nopcc_8, II_350_nopcc_9, II_350_nopcc_10]

II_350_pcc95_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc95/fold_1.csv')
II_350_pcc95_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc95/fold_2.csv')
II_350_pcc95_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc95/fold_3.csv')
II_350_pcc95_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc95/fold_4.csv')
II_350_pcc95_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc95/fold_5.csv')
II_350_pcc95_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc95/fold_6.csv')
II_350_pcc95_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc95/fold_7.csv')
II_350_pcc95_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc95/fold_8.csv')
II_350_pcc95_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc95/fold_9.csv')
II_350_pcc95_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc95/fold_10.csv')
datasets['II_350_pcc95'] = [II_350_pcc95_1, II_350_pcc95_2, II_350_pcc95_3, II_350_pcc95_4, II_350_pcc95_5,
                                       II_350_pcc95_6, II_350_pcc95_7, II_350_pcc95_8, II_350_pcc95_9, II_350_pcc95_10]

II_350_pcc75_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc75/fold_1.csv')
II_350_pcc75_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc75/fold_2.csv')
II_350_pcc75_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc75/fold_3.csv')
II_350_pcc75_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc75/fold_4.csv')
II_350_pcc75_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc75/fold_5.csv')
II_350_pcc75_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc75/fold_6.csv')
II_350_pcc75_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc75/fold_7.csv')
II_350_pcc75_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc75/fold_8.csv')
II_350_pcc75_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc75/fold_9.csv')
II_350_pcc75_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/II/350/cv/pcc75/fold_10.csv')
datasets['II_350_pcc75'] = [II_350_pcc75_1, II_350_pcc75_2, II_350_pcc75_3, II_350_pcc75_4, II_350_pcc75_5,
                                       II_350_pcc75_6, II_350_pcc75_7, II_350_pcc75_8, II_350_pcc75_9, II_350_pcc75_10]


In [8]:
def view_results_df(results_df, key):
    print("10-fold cross validation results for results_df with key:", key)
    for i in range(len(results_df)):
        # Format accuracy and MCC to 3 decimal places
        accuracy = f"{results_df['Accuracy'][i]:.3f}"
        mcc = f"{results_df['MCC'][i]:.3f}"
        print(f"Fold {i}, Accuracy: {accuracy}, MCC: {mcc}")


In [9]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [10]:
II_all_nopcc_results_df = train_with_base_ncart(datasets, 'II_all_nopcc')
print("\n---------------------------------------------------------------------\n")
II_all_pcc95_results_df = train_with_base_ncart(datasets, 'II_all_pcc95')
print("\n---------------------------------------------------------------------\n")
II_all_pcc75_results_df = train_with_base_ncart(datasets, 'II_all_pcc75')
print("\n---------------------------------------------------------------------\n")

Currently training NCART model with dataset key: II_all_nopcc
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.791399 |  Val Loss 0.763103
Epoch 1: Train Loss 0.765827 |  Val Loss 0.735707
Epoch 2: Train Loss 0.723946 |  Val Loss 0.711400
Epoch 3: Train Loss 0.714289 |  Val Loss 0.691388
Epoch 4: Train Loss 0.675458 |  Val Loss 0.675104
Epoch 5: Train Loss 0.653077 |  Val Loss 0.661260
Epoch 6: Train Loss 0.651520 |  Val Loss 0.650860
Epoch 7: Train Loss 0.628351 |  Val Loss 0.641851
Epoch 8: Train Loss 0.614033 |  Val Loss 0.632868
Epoch 9: Train Loss 0.612386 |  Val Loss 0.624428
Epoch 10: Train Loss 0.597089 |  Val Loss 0.616977
Epoch 11: Train Loss 0.591526 |  Val Loss 0.611446
Epoch 12: Train Loss 0.585476 |  Val Loss 0.606964
Epoch 13: Train Loss 0.575279 |  Val Loss 0.601464
Epoch 14: Train Loss 0.566794 |  Val Loss 0.596300
Epoch 15: Train Loss 0.555249 |  Val Loss 0.590726
Epoch 16: Train Loss 0.555582 |  Val Loss 0.583529
Epoch 17: Train Loss 0.546648 |  Val Los

In [11]:
II_550_nopcc_results_df = train_with_base_ncart(datasets, 'II_550_nopcc')
print("\n---------------------------------------------------------------------\n")
II_550_pcc95_results_df = train_with_base_ncart(datasets, 'II_550_pcc95')
print("\n---------------------------------------------------------------------\n")
II_550_pcc75_results_df = train_with_base_ncart(datasets, 'II_550_pcc75')
print("\n---------------------------------------------------------------------\n")

Streaming output truncated to the last 5000 lines.
Epoch 120: Train Loss 0.178582 |  Val Loss 0.314921
Epoch 121: Train Loss 0.175728 |  Val Loss 0.311307
Epoch 122: Train Loss 0.173743 |  Val Loss 0.311493
Epoch 123: Train Loss 0.171953 |  Val Loss 0.313436
Epoch 124: Train Loss 0.168728 |  Val Loss 0.313429
Epoch 125: Train Loss 0.167650 |  Val Loss 0.308065
Epoch 126: Train Loss 0.165325 |  Val Loss 0.304560
Epoch 127: Train Loss 0.163326 |  Val Loss 0.303908
Epoch 128: Train Loss 0.159944 |  Val Loss 0.305879
Epoch 129: Train Loss 0.159372 |  Val Loss 0.300796
Epoch 130: Train Loss 0.155880 |  Val Loss 0.298340
Epoch 131: Train Loss 0.154812 |  Val Loss 0.296255
Epoch 132: Train Loss 0.151205 |  Val Loss 0.298975
Epoch 133: Train Loss 0.150688 |  Val Loss 0.296756
Epoch 134: Train Loss 0.146997 |  Val Loss 0.295153
Epoch 135: Train Loss 0.146035 |  Val Loss 0.287980
Epoch 136: Train Loss 0.143474 |  Val Loss 0.282996
Epoch 137: Train Loss 0.141342 |  Val Loss 0.284811
Epoch 138: Tr

In [12]:
II_450_nopcc_results_df = train_with_base_ncart(datasets, 'II_450_nopcc')
print("\n---------------------------------------------------------------------\n")
II_450_pcc95_results_df = train_with_base_ncart(datasets, 'II_450_pcc95')
print("\n---------------------------------------------------------------------\n")
II_450_pcc75_results_df = train_with_base_ncart(datasets, 'II_450_pcc75')
print("\n---------------------------------------------------------------------\n")

Streaming output truncated to the last 5000 lines.
Epoch 122: Train Loss 0.170474 |  Val Loss 0.362566
Epoch 123: Train Loss 0.168028 |  Val Loss 0.363234
Epoch 124: Train Loss 0.165736 |  Val Loss 0.366595
Epoch 125: Train Loss 0.163555 |  Val Loss 0.364871
Epoch 126: Train Loss 0.161075 |  Val Loss 0.362737
Epoch 127: Train Loss 0.159011 |  Val Loss 0.366126
Epoch 128: Train Loss 0.156718 |  Val Loss 0.367057
Epoch 129: Train Loss 0.154452 |  Val Loss 0.363912
Epoch 130: Train Loss 0.152328 |  Val Loss 0.361536
Epoch 131: Train Loss 0.150274 |  Val Loss 0.363321
Epoch 132: Train Loss 0.148067 |  Val Loss 0.363957
Epoch 133: Train Loss 0.146117 |  Val Loss 0.364137
Epoch 134: Train Loss 0.144007 |  Val Loss 0.363458
Epoch 135: Train Loss 0.141998 |  Val Loss 0.362884
Epoch 136: Train Loss 0.140056 |  Val Loss 0.361073
Epoch 137: Train Loss 0.138068 |  Val Loss 0.360116
Epoch 138: Train Loss 0.136143 |  Val Loss 0.361952
Epoch 139: Train Loss 0.134286 |  Val Loss 0.363736
Epoch 140: Tr

In [13]:
II_350_nopcc_results_df = train_with_base_ncart(datasets, 'II_350_nopcc')
print("\n---------------------------------------------------------------------\n")
II_350_pcc95_results_df = train_with_base_ncart(datasets, 'II_350_pcc95')
print("\n---------------------------------------------------------------------\n")
II_350_pcc75_results_df = train_with_base_ncart(datasets, 'II_350_pcc75')

Streaming output truncated to the last 5000 lines.
Epoch 22: Train Loss 0.581072 |  Val Loss 0.593141
Epoch 23: Train Loss 0.575375 |  Val Loss 0.587966
Epoch 24: Train Loss 0.569856 |  Val Loss 0.582898
Epoch 25: Train Loss 0.564462 |  Val Loss 0.577741
Epoch 26: Train Loss 0.559181 |  Val Loss 0.572428
Epoch 27: Train Loss 0.554074 |  Val Loss 0.567499
Epoch 28: Train Loss 0.549103 |  Val Loss 0.562664
Epoch 29: Train Loss 0.544195 |  Val Loss 0.557928
Epoch 30: Train Loss 0.539383 |  Val Loss 0.553318
Epoch 31: Train Loss 0.534622 |  Val Loss 0.548758
Epoch 32: Train Loss 0.529917 |  Val Loss 0.544309
Epoch 33: Train Loss 0.525310 |  Val Loss 0.540054
Epoch 34: Train Loss 0.520786 |  Val Loss 0.535999
Epoch 35: Train Loss 0.516279 |  Val Loss 0.532013
Epoch 36: Train Loss 0.511881 |  Val Loss 0.527969
Epoch 37: Train Loss 0.507516 |  Val Loss 0.523828
Epoch 38: Train Loss 0.503182 |  Val Loss 0.519898
Epoch 39: Train Loss 0.498869 |  Val Loss 0.515674
Epoch 40: Train Loss 0.494455 |

In [14]:
print("Base Model:")
print('\n---------------------------------------------------------------------\n')
view_results_df(II_all_nopcc_results_df, 'II_all_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_all_pcc95_results_df, 'II_all_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_all_nopcc_results_df, 'II_all_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_550_nopcc_results_df, 'II_550_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_550_pcc95_results_df, 'II_550_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_550_nopcc_results_df, 'II_550_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_450_nopcc_results_df, 'II_450_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_450_pcc95_results_df, 'II_450_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_450_nopcc_results_df, 'II_450_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_350_nopcc_results_df, 'II_350_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_350_pcc95_results_df, 'II_350_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_350_nopcc_results_df, 'II_350_pcc75')
print('\n---------------------------------------------------------------------\n')


Base Model:

---------------------------------------------------------------------

10-fold cross validation results for results_df with key: II_all_nopcc
Fold 0, Accuracy: 0.898, MCC: 0.799
Fold 1, Accuracy: 0.820, MCC: 0.645
Fold 2, Accuracy: 0.829, MCC: 0.673
Fold 3, Accuracy: 0.857, MCC: 0.721
Fold 4, Accuracy: 0.808, MCC: 0.618
Fold 5, Accuracy: 0.849, MCC: 0.700
Fold 6, Accuracy: 0.873, MCC: 0.747
Fold 7, Accuracy: 0.861, MCC: 0.731
Fold 8, Accuracy: 0.857, MCC: 0.716
Fold 9, Accuracy: 0.788, MCC: 0.613

---------------------------------------------------------------------

10-fold cross validation results for results_df with key: II_all_pcc95
Fold 0, Accuracy: 0.837, MCC: 0.682
Fold 1, Accuracy: 0.849, MCC: 0.698
Fold 2, Accuracy: 0.849, MCC: 0.699
Fold 3, Accuracy: 0.857, MCC: 0.718
Fold 4, Accuracy: 0.816, MCC: 0.633
Fold 5, Accuracy: 0.788, MCC: 0.597
Fold 6, Accuracy: 0.878, MCC: 0.758
Fold 7, Accuracy: 0.873, MCC: 0.748
Fold 8, Accuracy: 0.861, MCC: 0.724
Fold 9, Accuracy: 

In [18]:
import time

start_time = time.time()
II_all_nopcc_results_df = train_with_best_hyperparameters(datasets, 'II_all_nopcc')
print("--- %s seconds ---" % (time.time() - start_time))

[I 2025-01-04 14:47:49,914] A new study created in memory with name: no-name-502dc64a-ddf8-4c8a-8e1a-05fe9928393b


Currently training NCART model with dataset key: II_all_nopcc
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.692571 |  Val Loss 0.689240
Epoch 1: Train Loss 0.685136 |  Val Loss 0.684550
Epoch 2: Train Loss 0.680566 |  Val Loss 0.679817
Epoch 3: Train Loss 0.674902 |  Val Loss 0.675189
Epoch 4: Train Loss 0.669933 |  Val Loss 0.671679
Epoch 5: Train Loss 0.664676 |  Val Loss 0.668397
Epoch 6: Train Loss 0.659329 |  Val Loss 0.664712
Epoch 7: Train Loss 0.652396 |  Val Loss 0.661161
Epoch 8: Train Loss 0.649168 |  Val Loss 0.657689
Epoch 9: Train Loss 0.644679 |  Val Loss 0.654263
Epoch 10: Train Loss 0.641571 |  Val Loss 0.650727
Epoch 11: Train Loss 0.638204 |  Val Loss 0.647631
Epoch 12: Train Loss 0.630402 |  Val Loss 0.642834
Epoch 13: Train Loss 0.624356 |  Val Loss 0.637697
Epoch 14: Train Loss 0.618604 |  Val Loss 0.633256
Epoch 15: Train Loss 0.614803 |  Val Loss 0.628137
Epoch 16: Train Loss 0.611235 |  Val Loss 0.622068
Epoch 17: Train Loss 0.605620 |  Val Los

[I 2025-01-04 14:47:57,798] Trial 0 finished with value: 0.6181396010295319 and parameters: {'n_trees': 56, 'n_selected': 4, 'n_layers': 6, 'mask_type': 'entmax'}. Best is trial 0 with value: 0.6181396010295319.


Epoch 126: Train Loss 0.117775 |  Val Loss 0.328763
Epoch 127: Train Loss 0.103354 |  Val Loss 0.347064
Epoch 128: Train Loss 0.096447 |  Val Loss 0.328631
Epoch 129: Train Loss 0.101343 |  Val Loss 0.297572
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.755296 |  Val Loss 0.719038
Epoch 1: Train Loss 0.704529 |  Val Loss 0.679599
Epoch 2: Train Loss 0.659739 |  Val Loss 0.654691
Epoch 3: Train Loss 0.639057 |  Val Loss 0.641680
Epoch 4: Train Loss 0.619118 |  Val Loss 0.634196
Epoch 5: Train Loss 0.607981 |  Val Loss 0.625957
Epoch 6: Train Loss 0.589282 |  Val Loss 0.616387
Epoch 7: Train Loss 0.569451 |  Val Loss 0.606368
Epoch 8: Train Loss 0.557737 |  Val Loss 0.594385
Epoch 9: Train Loss 0.542198 |  Val Loss 0.583579
Epoch 10: Train Loss 0.528594 |  Val Loss 0.575294
Epoch 11: Train Loss 0.525643 |  Val Loss 0.569609
Epoch 12: Train Loss 0.500127 |  Val Loss 0.559307
Epoch 13: Train Loss 0.4846

[I 2025-01-04 14:48:01,617] Trial 1 finished with value: 0.7662817146030789 and parameters: {'n_trees': 104, 'n_selected': 16, 'n_layers': 4, 'mask_type': 'sparsemax'}. Best is trial 1 with value: 0.7662817146030789.


...

Epoch 71: Train Loss 0.113812 |  Val Loss 0.322318
Epoch 72: Train Loss 0.120093 |  Val Loss 0.346620
Epoch 73: Train Loss 0.115483 |  Val Loss 0.335366
Epoch 74: Train Loss 0.102947 |  Val Loss 0.334942
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.728651 |  Val Loss 0.717910
Epoch 1: Train Loss 0.700055 |  Val Loss 0.698380
Epoch 2: Train Loss 0.688318 |  Val Loss 0.683964
Epoch 3: Train Loss 0.674171 |  Val Loss 0.673954
Epoch 4: Train Loss 0.665691 |  Val Loss 0.667047
Epoch 5: Train Loss 0.658202 |  Val Loss 0.662075
Epoch 6: Train Loss 0.652465 |  Val Loss 0.657948
Epoch 7: Train Loss 0.642701 |  Val Loss 0.653725
Epoch 8: Train Loss 0.640568 |  Val Loss 0.648448
Epoch 9: Train Loss 0.636854 |  Val Loss 0.642893
Epoch 10: Train Loss 0.626681 |  Val Loss 0.637827
Epoch 11: Train Loss 0.622946 |  Val Loss 0.633930
Epoch 12: Train Loss 0.617275 |  Val Loss 0.629815
Epoch 13: Train Loss 0.612902 |

[I 2025-01-04 14:49:03,328] Trial 13 finished with value: 0.7779495352862258 and parameters: {'n_trees': 80, 'n_selected': 12, 'n_layers': 2, 'mask_type': 'sparsemax'}. Best is trial 4 with value: 0.7890352271546528.


Epoch 135: Train Loss 0.128844 |  Val Loss 0.308754
Epoch 136: Train Loss 0.122427 |  Val Loss 0.321809
Epoch 137: Train Loss 0.142738 |  Val Loss 0.322596
Epoch 138: Train Loss 0.159311 |  Val Loss 0.281767
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.698909 |  Val Loss 0.692633
Epoch 1: Train Loss 0.693395 |  Val Loss 0.686646
Epoch 2: Train Loss 0.684355 |  Val Loss 0.682482
Epoch 3: Train Loss 0.680380 |  Val Loss 0.679271
Epoch 4: Train Loss 0.676687 |  Val Loss 0.676834
Epoch 5: Train Loss 0.674691 |  Val Loss 0.673851
Epoch 6: Train Loss 0.669017 |  Val Loss 0.670615
Epoch 7: Train Loss 0.662269 |  Val Loss 0.667189
Epoch 8: Train Loss 0.661681 |  Val Loss 0.664026
Epoch 9: Train Loss 0.657777 |  Val Loss 0.661129
Epoch 10: Train Loss 0.653284 |  Val Loss 0.658027
Epoch 11: Train Loss 0.649760 |  Val Loss 0.655222
Epoch 12: Train Loss 0.644874 |  Val Loss 0.652019
Epoch 13: Train Loss 0.6421

[I 2025-01-04 14:49:09,427] Trial 14 finished with value: 0.7405965210112124 and parameters: {'n_trees': 80, 'n_selected': 8, 'n_layers': 2, 'mask_type': 'entmax'}. Best is trial 4 with value: 0.7890352271546528.


Epoch 135: Train Loss 0.153992 |  Val Loss 0.288516
Epoch 136: Train Loss 0.146245 |  Val Loss 0.311446
Epoch 137: Train Loss 0.162222 |  Val Loss 0.315766
Epoch 138: Train Loss 0.168236 |  Val Loss 0.290179
Validation loss has not improved for 10 steps!
Early stopping applies.
Best hyperparameters: {'n_trees': 32, 'n_selected': 8, 'n_layers': 2, 'mask_type': 'entmax'}
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.693109 |  Val Loss 0.685490
Epoch 1: Train Loss 0.686576 |  Val Loss 0.678878
Epoch 2: Train Loss 0.675517 |  Val Loss 0.673834
Epoch 3: Train Loss 0.672987 |  Val Loss 0.670001
Epoch 4: Train Loss 0.666696 |  Val Loss 0.666868
Epoch 5: Train Loss 0.663944 |  Val Loss 0.664172
Epoch 6: Train Loss 0.660140 |  Val Loss 0.661380
Epoch 7: Train Loss 0.653534 |  Val Loss 0.658550
Epoch 8: Train Loss 0.650565 |  Val Loss 0.655807
Epoch 9: Train Loss 0.647507 |  Val Loss 0.653021
Epoch 10: Train Loss 0.645747 |  Val Loss 0.650079
Epoch 11: Train Loss 0.642885 |  Val

In [19]:
start_time = time.time()
II_all_pcc95_results_df = train_with_best_hyperparameters(datasets, 'II_all_pcc95')
print("--- %s seconds ---" % (time.time() - start_time))

[I 2025-01-04 14:50:16,453] A new study created in memory with name: no-name-0947fae7-6325-4877-8f2b-a53478209ddf


Currently training NCART model with dataset key: II_all_pcc95
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.701984 |  Val Loss 0.704280
Epoch 1: Train Loss 0.696304 |  Val Loss 0.702323
Epoch 2: Train Loss 0.699676 |  Val Loss 0.700623
Epoch 3: Train Loss 0.693621 |  Val Loss 0.699095
Epoch 4: Train Loss 0.696712 |  Val Loss 0.697805
Epoch 5: Train Loss 0.696179 |  Val Loss 0.696626
Epoch 6: Train Loss 0.691180 |  Val Loss 0.695622
Epoch 7: Train Loss 0.691981 |  Val Loss 0.694811
Epoch 8: Train Loss 0.691232 |  Val Loss 0.694133
Epoch 9: Train Loss 0.689721 |  Val Loss 0.693566
Epoch 10: Train Loss 0.689968 |  Val Loss 0.693106
Epoch 11: Train Loss 0.690802 |  Val Loss 0.692689
Epoch 12: Train Loss 0.688966 |  Val Loss 0.692311
Epoch 13: Train Loss 0.688839 |  Val Loss 0.691976
Epoch 14: Train Loss 0.688918 |  Val Loss 0.691660
Epoch 15: Train Loss 0.688290 |  Val Loss 0.691352
Epoch 16: Train Loss 0.687422 |  Val Loss 0.691054
Epoch 17: Train Loss 0.688414 |  Val Los

[I 2025-01-04 14:50:35,448] Trial 0 finished with value: 0.402709499389256 and parameters: {'n_trees': 112, 'n_selected': 4, 'n_layers': 1, 'mask_type': 'entmax'}. Best is trial 0 with value: 0.402709499389256.


Epoch 527: Train Loss 0.479577 |  Val Loss 0.577293
Epoch 528: Train Loss 0.463379 |  Val Loss 0.577073
Epoch 529: Train Loss 0.481763 |  Val Loss 0.576988
Epoch 530: Train Loss 0.489770 |  Val Loss 0.576955
Epoch 531: Train Loss 0.498859 |  Val Loss 0.577162
Epoch 532: Train Loss 0.489392 |  Val Loss 0.577322
Epoch 533: Train Loss 0.481329 |  Val Loss 0.577330
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.746628 |  Val Loss 0.712696
Epoch 1: Train Loss 0.689993 |  Val Loss 0.680979
Epoch 2: Train Loss 0.666590 |  Val Loss 0.667149
Epoch 3: Train Loss 0.658806 |  Val Loss 0.661681
Epoch 4: Train Loss 0.645160 |  Val Loss 0.654835
Epoch 5: Train Loss 0.635553 |  Val Loss 0.647396
Epoch 6: Train Loss 0.630111 |  Val Loss 0.638827
Epoch 7: Train Loss 0.609596 |  Val Loss 0.630502
Epoch 8: Train Loss 0.606096 |  Val Loss 0.624288
Epoch 9: Train Loss 0.598579 |  Val Loss 0.619448
Epoch 10: Train Loss 0.5

[I 2025-01-04 14:50:38,714] Trial 1 finished with value: 0.6770380081172879 and parameters: {'n_trees': 56, 'n_selected': 20, 'n_layers': 3, 'mask_type': 'entmax'}. Best is trial 1 with value: 0.6770380081172879.


...

Epoch 142: Train Loss 0.158812 |  Val Loss 0.317892
Epoch 143: Train Loss 0.163281 |  Val Loss 0.325487
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.700984 |  Val Loss 0.697281
Epoch 1: Train Loss 0.689042 |  Val Loss 0.689657
Epoch 2: Train Loss 0.681815 |  Val Loss 0.684299
Epoch 3: Train Loss 0.678572 |  Val Loss 0.680051
Epoch 4: Train Loss 0.671533 |  Val Loss 0.676386
Epoch 5: Train Loss 0.668708 |  Val Loss 0.672931
Epoch 6: Train Loss 0.663114 |  Val Loss 0.669673
Epoch 7: Train Loss 0.657033 |  Val Loss 0.666559
Epoch 8: Train Loss 0.655805 |  Val Loss 0.663532
Epoch 9: Train Loss 0.651061 |  Val Loss 0.660331
Epoch 10: Train Loss 0.644136 |  Val Loss 0.657114
Epoch 11: Train Loss 0.640579 |  Val Loss 0.653835
Epoch 12: Train Loss 0.637105 |  Val Loss 0.650631
Epoch 13: Train Loss 0.634633 |  Val Loss 0.647740
Epoch 14: Train Loss 0.630056 |  Val Loss 0.645299
Epoch 15: Train Loss 0.622916

[I 2025-01-04 14:51:34,990] Trial 13 finished with value: 0.7160390103945312 and parameters: {'n_trees': 40, 'n_selected': 8, 'n_layers': 2, 'mask_type': 'entmax'}. Best is trial 2 with value: 0.7407192072666816.


Epoch 166: Train Loss 0.152846 |  Val Loss 0.338508
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.748842 |  Val Loss 0.739124
Epoch 1: Train Loss 0.722591 |  Val Loss 0.722623
Epoch 2: Train Loss 0.715967 |  Val Loss 0.708372
Epoch 3: Train Loss 0.692991 |  Val Loss 0.696161
Epoch 4: Train Loss 0.687785 |  Val Loss 0.685221
Epoch 5: Train Loss 0.678000 |  Val Loss 0.675475
Epoch 6: Train Loss 0.660485 |  Val Loss 0.667019
Epoch 7: Train Loss 0.651110 |  Val Loss 0.659542
Epoch 8: Train Loss 0.644862 |  Val Loss 0.652756
Epoch 9: Train Loss 0.634610 |  Val Loss 0.646084
Epoch 10: Train Loss 0.626800 |  Val Loss 0.639727
Epoch 11: Train Loss 0.620442 |  Val Loss 0.633963
Epoch 12: Train Loss 0.608721 |  Val Loss 0.628150
Epoch 13: Train Loss 0.602192 |  Val Loss 0.621412
Epoch 14: Train Loss 0.595417 |  Val Loss 0.616220
Epoch 15: Train Loss 0.586264 |  Val Loss 0.611201
Epoch 16: Train Loss 0.582825 

[I 2025-01-04 14:51:39,161] Trial 14 finished with value: 0.6988621937318692 and parameters: {'n_trees': 80, 'n_selected': 8, 'n_layers': 4, 'mask_type': 'entmax'}. Best is trial 2 with value: 0.7407192072666816.


Epoch 78: Train Loss 0.136652 |  Val Loss 0.353200
Epoch 79: Train Loss 0.157187 |  Val Loss 0.366850
Epoch 80: Train Loss 0.112310 |  Val Loss 0.391319
Validation loss has not improved for 10 steps!
Early stopping applies.
Best hyperparameters: {'n_trees': 64, 'n_selected': 4, 'n_layers': 2, 'mask_type': 'entmax'}
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.692822 |  Val Loss 0.690627
Epoch 1: Train Loss 0.689962 |  Val Loss 0.688658
Epoch 2: Train Loss 0.687827 |  Val Loss 0.687130
Epoch 3: Train Loss 0.686357 |  Val Loss 0.685870
Epoch 4: Train Loss 0.683298 |  Val Loss 0.684543
Epoch 5: Train Loss 0.682587 |  Val Loss 0.683159
Epoch 6: Train Loss 0.681390 |  Val Loss 0.681875
Epoch 7: Train Loss 0.678553 |  Val Loss 0.680724
Epoch 8: Train Loss 0.677919 |  Val Loss 0.679682
Epoch 9: Train Loss 0.676166 |  Val Loss 0.678293
Epoch 10: Train Loss 0.674186 |  Val Loss 0.677148
Epoch 11: Train Loss 0.671967 |  Val Loss 0.675775
Epoch 12: Train Loss 0.670011 |  Val Los

In [20]:
start_time = time.time()
II_all_pcc75_results_df = train_with_best_hyperparameters(datasets, 'II_all_pcc75')
print("--- %s seconds ---" % (time.time() - start_time))

[I 2025-01-04 14:52:53,865] A new study created in memory with name: no-name-24ae93ea-23f7-41ee-9834-1ebbc8c1ad20


Currently training NCART model with dataset key: II_all_pcc75
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.961267 |  Val Loss 0.882346
Epoch 1: Train Loss 0.848384 |  Val Loss 0.784444
Epoch 2: Train Loss 0.731665 |  Val Loss 0.714782
Epoch 3: Train Loss 0.680666 |  Val Loss 0.668337
Epoch 4: Train Loss 0.619397 |  Val Loss 0.643378
Epoch 5: Train Loss 0.607861 |  Val Loss 0.633542
Epoch 6: Train Loss 0.588870 |  Val Loss 0.629310
Epoch 7: Train Loss 0.573726 |  Val Loss 0.624211
Epoch 8: Train Loss 0.573428 |  Val Loss 0.616245
Epoch 9: Train Loss 0.557629 |  Val Loss 0.606879
Epoch 10: Train Loss 0.538462 |  Val Loss 0.597266
Epoch 11: Train Loss 0.541400 |  Val Loss 0.585887
Epoch 12: Train Loss 0.513661 |  Val Loss 0.576047
Epoch 13: Train Loss 0.505009 |  Val Loss 0.571371
Epoch 14: Train Loss 0.499650 |  Val Loss 0.565841
Epoch 15: Train Loss 0.488881 |  Val Loss 0.557933
Epoch 16: Train Loss 0.485441 |  Val Loss 0.551017
Epoch 17: Train Loss 0.476362 |  Val Los

[I 2025-01-04 14:52:57,007] Trial 0 finished with value: 0.6762285840668529 and parameters: {'n_trees': 40, 'n_selected': 20, 'n_layers': 4, 'mask_type': 'sparsemax'}. Best is trial 0 with value: 0.6762285840668529.


Epoch 65: Train Loss 0.172217 |  Val Loss 0.427659
Epoch 66: Train Loss 0.153202 |  Val Loss 0.421184
Epoch 67: Train Loss 0.143938 |  Val Loss 0.394625
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 1.087301 |  Val Loss 0.901553
Epoch 1: Train Loss 0.823602 |  Val Loss 0.723666
Epoch 2: Train Loss 0.689754 |  Val Loss 0.660360
Epoch 3: Train Loss 0.655004 |  Val Loss 0.669347
Epoch 4: Train Loss 0.653474 |  Val Loss 0.675914
Epoch 5: Train Loss 0.647784 |  Val Loss 0.657332
Epoch 6: Train Loss 0.635049 |  Val Loss 0.627827
Epoch 7: Train Loss 0.582901 |  Val Loss 0.604221
Epoch 8: Train Loss 0.567353 |  Val Loss 0.592725
Epoch 9: Train Loss 0.550506 |  Val Loss 0.587551
Epoch 10: Train Loss 0.534840 |  Val Loss 0.581190
Epoch 11: Train Loss 0.532586 |  Val Loss 0.568912
Epoch 12: Train Loss 0.497548 |  Val Loss 0.556480
Epoch 13: Train Loss 0.487236 |  Val Loss 0.546921
Epoch 14: Train Loss 0.474391 |

[I 2025-01-04 14:53:00,022] Trial 1 finished with value: 0.627093443885227 and parameters: {'n_trees': 128, 'n_selected': 32, 'n_layers': 4, 'mask_type': 'entmax'}. Best is trial 0 with value: 0.6762285840668529.


...

Epoch 255: Train Loss 0.255183 |  Val Loss 0.407909
Epoch 256: Train Loss 0.223730 |  Val Loss 0.406995
Epoch 257: Train Loss 0.232495 |  Val Loss 0.404763
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.699275 |  Val Loss 0.692497
Epoch 1: Train Loss 0.688747 |  Val Loss 0.685265
Epoch 2: Train Loss 0.681991 |  Val Loss 0.679836
Epoch 3: Train Loss 0.676050 |  Val Loss 0.675774
Epoch 4: Train Loss 0.669615 |  Val Loss 0.671654
Epoch 5: Train Loss 0.666021 |  Val Loss 0.667316
Epoch 6: Train Loss 0.660866 |  Val Loss 0.663075
Epoch 7: Train Loss 0.652388 |  Val Loss 0.658707
Epoch 8: Train Loss 0.649760 |  Val Loss 0.654457
Epoch 9: Train Loss 0.644588 |  Val Loss 0.650469
Epoch 10: Train Loss 0.635163 |  Val Loss 0.646208
Epoch 11: Train Loss 0.633186 |  Val Loss 0.641537
Epoch 12: Train Loss 0.622579 |  Val Loss 0.636508
Epoch 13: Train Loss 0.617893 |  Val Loss 0.631606
Epoch 14: Train Loss 0.61097

[I 2025-01-04 14:54:07,780] Trial 13 finished with value: 0.6829494005118091 and parameters: {'n_trees': 64, 'n_selected': 8, 'n_layers': 5, 'mask_type': 'entmax'}. Best is trial 8 with value: 0.7096663458578801.


Epoch 77: Train Loss 0.131346 |  Val Loss 0.382262
Epoch 78: Train Loss 0.126824 |  Val Loss 0.386413
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.683512 |  Val Loss 0.654083
Epoch 1: Train Loss 0.656499 |  Val Loss 0.648949
Epoch 2: Train Loss 0.649288 |  Val Loss 0.634587
Epoch 3: Train Loss 0.630527 |  Val Loss 0.623662
Epoch 4: Train Loss 0.610301 |  Val Loss 0.616375
Epoch 5: Train Loss 0.602195 |  Val Loss 0.608642
Epoch 6: Train Loss 0.593177 |  Val Loss 0.600935
Epoch 7: Train Loss 0.574280 |  Val Loss 0.594488
Epoch 8: Train Loss 0.572315 |  Val Loss 0.588683
Epoch 9: Train Loss 0.563520 |  Val Loss 0.582470
Epoch 10: Train Loss 0.539092 |  Val Loss 0.576755
Epoch 11: Train Loss 0.540630 |  Val Loss 0.572370
Epoch 12: Train Loss 0.521513 |  Val Loss 0.566948
Epoch 13: Train Loss 0.513954 |  Val Loss 0.560420
Epoch 14: Train Loss 0.507725 |  Val Loss 0.556374
Epoch 15: Train Loss 0.498501 |

[I 2025-01-04 14:54:11,085] Trial 14 finished with value: 0.6805794059787579 and parameters: {'n_trees': 88, 'n_selected': 28, 'n_layers': 3, 'mask_type': 'entmax'}. Best is trial 8 with value: 0.7096663458578801.


Epoch 67: Train Loss 0.166847 |  Val Loss 0.424165
Epoch 68: Train Loss 0.158898 |  Val Loss 0.397223
Validation loss has not improved for 10 steps!
Early stopping applies.
Best hyperparameters: {'n_trees': 88, 'n_selected': 16, 'n_layers': 3, 'mask_type': 'entmax'}
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.688191 |  Val Loss 0.674175
Epoch 1: Train Loss 0.673009 |  Val Loss 0.664688
Epoch 2: Train Loss 0.662104 |  Val Loss 0.655310
Epoch 3: Train Loss 0.653717 |  Val Loss 0.647595
Epoch 4: Train Loss 0.643383 |  Val Loss 0.640191
Epoch 5: Train Loss 0.635765 |  Val Loss 0.632926
Epoch 6: Train Loss 0.626363 |  Val Loss 0.626484
Epoch 7: Train Loss 0.613035 |  Val Loss 0.619500
Epoch 8: Train Loss 0.609260 |  Val Loss 0.613557
Epoch 9: Train Loss 0.602031 |  Val Loss 0.607900
Epoch 10: Train Loss 0.586231 |  Val Loss 0.601812
Epoch 11: Train Loss 0.583158 |  Val Loss 0.596016
Epoch 12: Train Loss 0.569178 |  Val Loss 0.590657
Epoch 13: Train Loss 0.561783 |  Val Lo

In [21]:
start_time = time.time()
II_550_nopcc_results_df = train_with_best_hyperparameters(datasets, 'II_550_nopcc')
print("--- %s seconds ---" % (time.time() - start_time))

[I 2025-01-04 14:54:48,086] A new study created in memory with name: no-name-5a1a5d20-bed3-4965-8a1e-01cb6eb81d98


Currently training NCART model with dataset key: II_550_nopcc
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.747985 |  Val Loss 0.718554
Epoch 1: Train Loss 0.713826 |  Val Loss 0.692232
Epoch 2: Train Loss 0.688141 |  Val Loss 0.672870
Epoch 3: Train Loss 0.668998 |  Val Loss 0.657978
Epoch 4: Train Loss 0.654893 |  Val Loss 0.647649
Epoch 5: Train Loss 0.645160 |  Val Loss 0.640010
Epoch 6: Train Loss 0.637764 |  Val Loss 0.633052
Epoch 7: Train Loss 0.631459 |  Val Loss 0.626045
Epoch 8: Train Loss 0.625013 |  Val Loss 0.618955
Epoch 9: Train Loss 0.618106 |  Val Loss 0.611455
Epoch 10: Train Loss 0.610624 |  Val Loss 0.603772
Epoch 11: Train Loss 0.602918 |  Val Loss 0.595958
Epoch 12: Train Loss 0.594931 |  Val Loss 0.588040
Epoch 13: Train Loss 0.586900 |  Val Loss 0.580559
Epoch 14: Train Loss 0.579142 |  Val Loss 0.573852
Epoch 15: Train Loss 0.571704 |  Val Loss 0.567580
Epoch 16: Train Loss 0.564628 |  Val Loss 0.561422
Epoch 17: Train Loss 0.557915 |  Val Los

[I 2025-01-04 14:54:50,374] Trial 0 finished with value: 0.7668115805072325 and parameters: {'n_trees': 96, 'n_selected': 24, 'n_layers': 3, 'mask_type': 'entmax'}. Best is trial 0 with value: 0.7668115805072325.


Epoch 89: Train Loss 0.064199 |  Val Loss 0.298933
Epoch 90: Train Loss 0.061752 |  Val Loss 0.299720
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.961952 |  Val Loss 0.923976
Epoch 1: Train Loss 0.924094 |  Val Loss 0.892391
Epoch 2: Train Loss 0.890900 |  Val Loss 0.866603
Epoch 3: Train Loss 0.862718 |  Val Loss 0.841908
Epoch 4: Train Loss 0.837517 |  Val Loss 0.820291
Epoch 5: Train Loss 0.814236 |  Val Loss 0.797914
Epoch 6: Train Loss 0.792529 |  Val Loss 0.777012
Epoch 7: Train Loss 0.771630 |  Val Loss 0.757027
Epoch 8: Train Loss 0.751230 |  Val Loss 0.738659
Epoch 9: Train Loss 0.731335 |  Val Loss 0.721959
Epoch 10: Train Loss 0.712686 |  Val Loss 0.706376
Epoch 11: Train Loss 0.695158 |  Val Loss 0.691115
Epoch 12: Train Loss 0.678939 |  Val Loss 0.675417
Epoch 13: Train Loss 0.662988 |  Val Loss 0.660654
Epoch 14: Train Loss 0.647278 |  Val Loss 0.647264
Epoch 15: Train Loss 0.632290 |

[I 2025-01-04 14:54:53,323] Trial 1 finished with value: 0.8011927448021527 and parameters: {'n_trees': 24, 'n_selected': 12, 'n_layers': 4, 'mask_type': 'sparsemax'}. Best is trial 1 with value: 0.8011927448021527.


...

Epoch 75: Train Loss 0.098116 |  Val Loss 0.367261
Epoch 76: Train Loss 0.094954 |  Val Loss 0.358717
Epoch 77: Train Loss 0.091678 |  Val Loss 0.353522
Epoch 78: Train Loss 0.088950 |  Val Loss 0.353555
Epoch 79: Train Loss 0.085859 |  Val Loss 0.350982
Epoch 80: Train Loss 0.083817 |  Val Loss 0.349401
Epoch 81: Train Loss 0.081981 |  Val Loss 0.351051
Epoch 82: Train Loss 0.079403 |  Val Loss 0.350942
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 1.163061 |  Val Loss 1.122078
Epoch 1: Train Loss 1.130270 |  Val Loss 1.088713
Epoch 2: Train Loss 1.100351 |  Val Loss 1.061515
Epoch 3: Train Loss 1.071872 |  Val Loss 1.035262
Epoch 4: Train Loss 1.045103 |  Val Loss 1.010083
Epoch 5: Train Loss 1.019571 |  Val Loss 0.987908
Epoch 6: Train Loss 0.995392 |  Val Loss 0.965607
Epoch 7: Train Loss 0.972814 |  Val Loss 0.945260
Epoch 8: Train Loss 0.951398 |  Val Loss 0.925429
Epoch 9: Train Loss 0.930907 |

[I 2025-01-04 14:55:34,101] Trial 13 finished with value: 0.7133483310547061 and parameters: {'n_trees': 8, 'n_selected': 20, 'n_layers': 5, 'mask_type': 'sparsemax'}. Best is trial 1 with value: 0.8011927448021527.


Epoch 156: Train Loss 0.055509 |  Val Loss 0.304841
Epoch 157: Train Loss 0.054580 |  Val Loss 0.306070
Epoch 158: Train Loss 0.053686 |  Val Loss 0.308203
Epoch 159: Train Loss 0.052798 |  Val Loss 0.309502
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.687309 |  Val Loss 0.672162
Epoch 1: Train Loss 0.669685 |  Val Loss 0.657185
Epoch 2: Train Loss 0.656565 |  Val Loss 0.646309
Epoch 3: Train Loss 0.646144 |  Val Loss 0.635922
Epoch 4: Train Loss 0.636371 |  Val Loss 0.626918
Epoch 5: Train Loss 0.626587 |  Val Loss 0.618446
Epoch 6: Train Loss 0.617031 |  Val Loss 0.610738
Epoch 7: Train Loss 0.607944 |  Val Loss 0.603175
Epoch 8: Train Loss 0.599071 |  Val Loss 0.595387
Epoch 9: Train Loss 0.590327 |  Val Loss 0.587394
Epoch 10: Train Loss 0.581471 |  Val Loss 0.579001
Epoch 11: Train Loss 0.572352 |  Val Loss 0.570778
Epoch 12: Train Loss 0.563083 |  Val Loss 0.562743
Epoch 13: Train Loss 0.5537

[I 2025-01-04 14:55:36,714] Trial 14 finished with value: 0.709560194985489 and parameters: {'n_trees': 48, 'n_selected': 12, 'n_layers': 6, 'mask_type': 'entmax'}. Best is trial 1 with value: 0.8011927448021527.


Epoch 83: Train Loss 0.104756 |  Val Loss 0.366441
Epoch 84: Train Loss 0.102297 |  Val Loss 0.367230
Epoch 85: Train Loss 0.099935 |  Val Loss 0.366617
Validation loss has not improved for 10 steps!
Early stopping applies.
Best hyperparameters: {'n_trees': 24, 'n_selected': 12, 'n_layers': 4, 'mask_type': 'sparsemax'}
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.961952 |  Val Loss 0.923976
Epoch 1: Train Loss 0.924094 |  Val Loss 0.892391
Epoch 2: Train Loss 0.890900 |  Val Loss 0.866603
Epoch 3: Train Loss 0.862718 |  Val Loss 0.841908
Epoch 4: Train Loss 0.837517 |  Val Loss 0.820291
Epoch 5: Train Loss 0.814236 |  Val Loss 0.797914
Epoch 6: Train Loss 0.792529 |  Val Loss 0.777012
Epoch 7: Train Loss 0.771630 |  Val Loss 0.757027
Epoch 8: Train Loss 0.751230 |  Val Loss 0.738659
Epoch 9: Train Loss 0.731335 |  Val Loss 0.721959
Epoch 10: Train Loss 0.712686 |  Val Loss 0.706376
Epoch 11: Train Loss 0.695158 |  Val Loss 0.691115
Epoch 12: Train Loss 0.678939 |  Val

In [22]:
start_time = time.time()
II_550_pcc95_results_df = train_with_best_hyperparameters(datasets, 'II_550_pcc95')
print("--- %s seconds ---" % (time.time() - start_time))

[I 2025-01-04 14:56:05,264] A new study created in memory with name: no-name-1ea7816f-89fd-4835-ba48-d2738d8fe77c


Currently training NCART model with dataset key: II_550_pcc95
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.827556 |  Val Loss 0.796507
Epoch 1: Train Loss 0.791656 |  Val Loss 0.762224
Epoch 2: Train Loss 0.759840 |  Val Loss 0.732173
Epoch 3: Train Loss 0.732154 |  Val Loss 0.704709
Epoch 4: Train Loss 0.707626 |  Val Loss 0.678837
Epoch 5: Train Loss 0.685926 |  Val Loss 0.655234
Epoch 6: Train Loss 0.666492 |  Val Loss 0.634125
Epoch 7: Train Loss 0.648334 |  Val Loss 0.615691
Epoch 8: Train Loss 0.631715 |  Val Loss 0.600012
Epoch 9: Train Loss 0.616278 |  Val Loss 0.586160
Epoch 10: Train Loss 0.602614 |  Val Loss 0.574586
Epoch 11: Train Loss 0.590241 |  Val Loss 0.563525
Epoch 12: Train Loss 0.579030 |  Val Loss 0.554279
Epoch 13: Train Loss 0.568554 |  Val Loss 0.546597
Epoch 14: Train Loss 0.558580 |  Val Loss 0.539593
Epoch 15: Train Loss 0.549014 |  Val Loss 0.532832
Epoch 16: Train Loss 0.539808 |  Val Loss 0.526693
Epoch 17: Train Loss 0.530969 |  Val Los

[I 2025-01-04 14:56:07,465] Trial 0 finished with value: 0.745947897292437 and parameters: {'n_trees': 32, 'n_selected': 16, 'n_layers': 7, 'mask_type': 'sparsemax'}. Best is trial 0 with value: 0.745947897292437.


Epoch 80: Train Loss 0.079354 |  Val Loss 0.328397
Epoch 81: Train Loss 0.076886 |  Val Loss 0.328838
Epoch 82: Train Loss 0.074815 |  Val Loss 0.329644
Epoch 83: Train Loss 0.072827 |  Val Loss 0.331595
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.730845 |  Val Loss 0.730565
Epoch 1: Train Loss 0.725551 |  Val Loss 0.725134
Epoch 2: Train Loss 0.720826 |  Val Loss 0.719046
Epoch 3: Train Loss 0.716395 |  Val Loss 0.712300
Epoch 4: Train Loss 0.712823 |  Val Loss 0.707856
Epoch 5: Train Loss 0.709744 |  Val Loss 0.704238
Epoch 6: Train Loss 0.706174 |  Val Loss 0.701056
Epoch 7: Train Loss 0.702608 |  Val Loss 0.698506
Epoch 8: Train Loss 0.699380 |  Val Loss 0.696280
Epoch 9: Train Loss 0.696469 |  Val Loss 0.693335
Epoch 10: Train Loss 0.693553 |  Val Loss 0.690318
Epoch 11: Train Loss 0.690748 |  Val Loss 0.687349
Epoch 12: Train Loss 0.687981 |  Val Loss 0.684396
Epoch 13: Train Loss 0.685237 |

[I 2025-01-04 14:56:13,282] Trial 1 finished with value: 0.7474338507517468 and parameters: {'n_trees': 48, 'n_selected': 4, 'n_layers': 3, 'mask_type': 'sparsemax'}. Best is trial 1 with value: 0.7474338507517468.


...

Epoch 71: Train Loss 0.050382 |  Val Loss 0.344971
Epoch 72: Train Loss 0.047804 |  Val Loss 0.346040
Epoch 73: Train Loss 0.045304 |  Val Loss 0.349951
Epoch 74: Train Loss 0.043058 |  Val Loss 0.351427
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.845890 |  Val Loss 0.768490
Epoch 1: Train Loss 0.785874 |  Val Loss 0.718877
Epoch 2: Train Loss 0.735932 |  Val Loss 0.676782
Epoch 3: Train Loss 0.692756 |  Val Loss 0.642588
Epoch 4: Train Loss 0.656419 |  Val Loss 0.615231
Epoch 5: Train Loss 0.627320 |  Val Loss 0.593173
Epoch 6: Train Loss 0.604346 |  Val Loss 0.576317
Epoch 7: Train Loss 0.586773 |  Val Loss 0.564045
Epoch 8: Train Loss 0.573380 |  Val Loss 0.555777
Epoch 9: Train Loss 0.563067 |  Val Loss 0.549463
Epoch 10: Train Loss 0.554738 |  Val Loss 0.544089
Epoch 11: Train Loss 0.547309 |  Val Loss 0.539329
Epoch 12: Train Loss 0.539916 |  Val Loss 0.534109
Epoch 13: Train Loss 0.531617 |

[I 2025-01-04 14:57:13,227] Trial 13 finished with value: 0.8001322641986387 and parameters: {'n_trees': 48, 'n_selected': 28, 'n_layers': 6, 'mask_type': 'entmax'}. Best is trial 2 with value: 0.8568144142763927.


Epoch 70: Train Loss 0.049200 |  Val Loss 0.342999
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.798048 |  Val Loss 0.721529
Epoch 1: Train Loss 0.748452 |  Val Loss 0.681648
Epoch 2: Train Loss 0.705768 |  Val Loss 0.649469
Epoch 3: Train Loss 0.670388 |  Val Loss 0.622415
Epoch 4: Train Loss 0.642272 |  Val Loss 0.602268
Epoch 5: Train Loss 0.620665 |  Val Loss 0.587065
Epoch 6: Train Loss 0.604273 |  Val Loss 0.576195
Epoch 7: Train Loss 0.591816 |  Val Loss 0.568000
Epoch 8: Train Loss 0.582377 |  Val Loss 0.562003
Epoch 9: Train Loss 0.574741 |  Val Loss 0.556132
Epoch 10: Train Loss 0.567743 |  Val Loss 0.550423
Epoch 11: Train Loss 0.560593 |  Val Loss 0.544154
Epoch 12: Train Loss 0.552400 |  Val Loss 0.536435
Epoch 13: Train Loss 0.543018 |  Val Loss 0.527270
Epoch 14: Train Loss 0.532369 |  Val Loss 0.517127
Epoch 15: Train Loss 0.520711 |  Val Loss 0.505971
Epoch 16: Train Loss 0.508399 |

[I 2025-01-04 14:57:15,535] Trial 14 finished with value: 0.8181818181818182 and parameters: {'n_trees': 88, 'n_selected': 24, 'n_layers': 7, 'mask_type': 'entmax'}. Best is trial 2 with value: 0.8568144142763927.


Epoch 61: Train Loss 0.062042 |  Val Loss 0.265409
Epoch 62: Train Loss 0.059120 |  Val Loss 0.260500
Epoch 63: Train Loss 0.056745 |  Val Loss 0.259181
Epoch 64: Train Loss 0.054385 |  Val Loss 0.257248
Epoch 65: Train Loss 0.052114 |  Val Loss 0.258178
Validation loss has not improved for 10 steps!
Early stopping applies.
Best hyperparameters: {'n_trees': 40, 'n_selected': 20, 'n_layers': 5, 'mask_type': 'sparsemax'}
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.874072 |  Val Loss 0.845591
Epoch 1: Train Loss 0.824007 |  Val Loss 0.801413
Epoch 2: Train Loss 0.779322 |  Val Loss 0.759247
Epoch 3: Train Loss 0.738438 |  Val Loss 0.720048
Epoch 4: Train Loss 0.701670 |  Val Loss 0.685039
Epoch 5: Train Loss 0.668866 |  Val Loss 0.654284
Epoch 6: Train Loss 0.639945 |  Val Loss 0.628317
Epoch 7: Train Loss 0.614482 |  Val Loss 0.606462
Epoch 8: Train Loss 0.592279 |  Val Loss 0.587845
Epoch 9: Train Loss 0.572827 |  Val Loss 0.571509
Epoch 10: Train Loss 0.556263 |  Val

In [23]:
start_time = time.time()
II_550_pcc75_results_df = train_with_best_hyperparameters(datasets, 'II_550_pcc75')
print("--- %s seconds ---" % (time.time() - start_time))

[I 2025-01-04 14:57:37,031] A new study created in memory with name: no-name-a074bcd6-7588-453c-bcb5-c57463443736


Currently training NCART model with dataset key: II_550_pcc75
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.769399 |  Val Loss 0.700775
Epoch 1: Train Loss 0.717653 |  Val Loss 0.662602
Epoch 2: Train Loss 0.680714 |  Val Loss 0.639666
Epoch 3: Train Loss 0.658931 |  Val Loss 0.630448
Epoch 4: Train Loss 0.650759 |  Val Loss 0.631003
Epoch 5: Train Loss 0.652232 |  Val Loss 0.635835
Epoch 6: Train Loss 0.657759 |  Val Loss 0.639982
Epoch 7: Train Loss 0.662314 |  Val Loss 0.640723
Epoch 8: Train Loss 0.663165 |  Val Loss 0.637620
Epoch 9: Train Loss 0.659908 |  Val Loss 0.631634
Epoch 10: Train Loss 0.653550 |  Val Loss 0.624276
Epoch 11: Train Loss 0.645648 |  Val Loss 0.617034
Epoch 12: Train Loss 0.637752 |  Val Loss 0.611077
Epoch 13: Train Loss 0.631073 |  Val Loss 0.607055
Epoch 14: Train Loss 0.626307 |  Val Loss 0.605013
Epoch 15: Train Loss 0.623560 |  Val Loss 0.604485
Epoch 16: Train Loss 0.622406 |  Val Loss 0.604693
Epoch 17: Train Loss 0.622070 |  Val Los

[I 2025-01-04 14:57:53,264] Trial 0 finished with value: 0.745947897292437 and parameters: {'n_trees': 104, 'n_selected': 32, 'n_layers': 1, 'mask_type': 'entmax'}. Best is trial 0 with value: 0.745947897292437.


Epoch 988: Train Loss 0.103796 |  Val Loss 0.271463
Epoch 989: Train Loss 0.103486 |  Val Loss 0.271408
Epoch 990: Train Loss 0.103178 |  Val Loss 0.271345
Epoch 991: Train Loss 0.102871 |  Val Loss 0.271265
Epoch 992: Train Loss 0.102563 |  Val Loss 0.271245
Epoch 993: Train Loss 0.102255 |  Val Loss 0.271177
Epoch 994: Train Loss 0.101950 |  Val Loss 0.271061
Epoch 995: Train Loss 0.101645 |  Val Loss 0.270951
Epoch 996: Train Loss 0.101341 |  Val Loss 0.270885
Epoch 997: Train Loss 0.101040 |  Val Loss 0.270860
Epoch 998: Train Loss 0.100738 |  Val Loss 0.270831
Epoch 999: Train Loss 0.100436 |  Val Loss 0.270797
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.715736 |  Val Loss 0.715469
Epoch 1: Train Loss 0.707591 |  Val Loss 0.706250
Epoch 2: Train Loss 0.700059 |  Val Loss 0.698224
Epoch 3: Train Loss 0.693606 |  Val Loss 0.692910
Epoch 4: Train Loss 0.687748 |  Val Loss 0.687802
Epoch 5: Train Loss 0.682527 |  Val Loss 0.683377
Epoch 6: Train Loss 0.677913 |  Val

[I 2025-01-04 14:57:58,038] Trial 1 finished with value: 0.7823355995993853 and parameters: {'n_trees': 32, 'n_selected': 12, 'n_layers': 2, 'mask_type': 'sparsemax'}. Best is trial 1 with value: 0.7823355995993853.


...

Epoch 97: Train Loss 0.045357 |  Val Loss 0.317506
Epoch 98: Train Loss 0.044058 |  Val Loss 0.317013
Epoch 99: Train Loss 0.042819 |  Val Loss 0.318520
Epoch 100: Train Loss 0.041668 |  Val Loss 0.319986
Epoch 101: Train Loss 0.040526 |  Val Loss 0.320495
Epoch 102: Train Loss 0.039435 |  Val Loss 0.320339
Epoch 103: Train Loss 0.038418 |  Val Loss 0.320074
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.771351 |  Val Loss 0.745257
Epoch 1: Train Loss 0.729744 |  Val Loss 0.716444
Epoch 2: Train Loss 0.697102 |  Val Loss 0.694931
Epoch 3: Train Loss 0.673622 |  Val Loss 0.680534
Epoch 4: Train Loss 0.657678 |  Val Loss 0.672049
Epoch 5: Train Loss 0.648314 |  Val Loss 0.669194
Epoch 6: Train Loss 0.643964 |  Val Loss 0.668627
Epoch 7: Train Loss 0.642146 |  Val Loss 0.666673
Epoch 8: Train Loss 0.640229 |  Val Loss 0.663738
Epoch 9: Train Loss 0.636647 |  Val Loss 0.658440
Epoch 10: Train Loss 0.6311

[I 2025-01-04 14:58:52,930] Trial 13 finished with value: 0.7167132003459626 and parameters: {'n_trees': 80, 'n_selected': 28, 'n_layers': 3, 'mask_type': 'entmax'}. Best is trial 5 with value: 0.820354226434844.


Epoch 106: Train Loss 0.073370 |  Val Loss 0.307914
Epoch 107: Train Loss 0.071084 |  Val Loss 0.313881
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.752077 |  Val Loss 0.737070
Epoch 1: Train Loss 0.733277 |  Val Loss 0.720251
Epoch 2: Train Loss 0.716528 |  Val Loss 0.705938
Epoch 3: Train Loss 0.702092 |  Val Loss 0.693292
Epoch 4: Train Loss 0.689587 |  Val Loss 0.681744
Epoch 5: Train Loss 0.678306 |  Val Loss 0.671474
Epoch 6: Train Loss 0.668147 |  Val Loss 0.662842
Epoch 7: Train Loss 0.659373 |  Val Loss 0.654851
Epoch 8: Train Loss 0.651852 |  Val Loss 0.647879
Epoch 9: Train Loss 0.645376 |  Val Loss 0.642189
Epoch 10: Train Loss 0.639827 |  Val Loss 0.637123
Epoch 11: Train Loss 0.635220 |  Val Loss 0.632817
Epoch 12: Train Loss 0.631212 |  Val Loss 0.628775
Epoch 13: Train Loss 0.627578 |  Val Loss 0.624956
Epoch 14: Train Loss 0.624284 |  Val Loss 0.621703
Epoch 15: Train Loss 0.621129

[I 2025-01-04 14:58:56,611] Trial 14 finished with value: 0.7818181818181819 and parameters: {'n_trees': 128, 'n_selected': 16, 'n_layers': 4, 'mask_type': 'entmax'}. Best is trial 5 with value: 0.820354226434844.


Epoch 146: Train Loss 0.044414 |  Val Loss 0.312934
Epoch 147: Train Loss 0.043552 |  Val Loss 0.311039
Epoch 148: Train Loss 0.042707 |  Val Loss 0.313057
Epoch 149: Train Loss 0.041807 |  Val Loss 0.314945
Epoch 150: Train Loss 0.041052 |  Val Loss 0.311587
Validation loss has not improved for 10 steps!
Early stopping applies.
Best hyperparameters: {'n_trees': 128, 'n_selected': 20, 'n_layers': 2, 'mask_type': 'entmax'}
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.752966 |  Val Loss 0.723261
Epoch 1: Train Loss 0.730538 |  Val Loss 0.705465
Epoch 2: Train Loss 0.711347 |  Val Loss 0.690516
Epoch 3: Train Loss 0.695168 |  Val Loss 0.678701
Epoch 4: Train Loss 0.682000 |  Val Loss 0.669264
Epoch 5: Train Loss 0.671332 |  Val Loss 0.661917
Epoch 6: Train Loss 0.662802 |  Val Loss 0.655992
Epoch 7: Train Loss 0.656140 |  Val Loss 0.651327
Epoch 8: Train Loss 0.650984 |  Val Loss 0.647541
Epoch 9: Train Loss 0.646828 |  Val Loss 0.644122
Epoch 10: Train Loss 0.643368 |  

In [24]:
start_time = time.time()
II_450_nopcc_results_df = train_with_best_hyperparameters(datasets, 'II_450_nopcc')
print("--- %s seconds ---" % (time.time() - start_time))

[I 2025-01-04 14:59:36,708] A new study created in memory with name: no-name-648e02d7-96e7-464f-80d9-6febe2bf8e75


Currently training NCART model with dataset key: II_450_nopcc
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 1.119767 |  Val Loss 1.123407
Epoch 1: Train Loss 1.107228 |  Val Loss 1.111063
Epoch 2: Train Loss 1.094882 |  Val Loss 1.098884
Epoch 3: Train Loss 1.082713 |  Val Loss 1.086904
Epoch 4: Train Loss 1.070729 |  Val Loss 1.075136
Epoch 5: Train Loss 1.058937 |  Val Loss 1.063536
Epoch 6: Train Loss 1.047335 |  Val Loss 1.052111
Epoch 7: Train Loss 1.035924 |  Val Loss 1.040874
Epoch 8: Train Loss 1.024710 |  Val Loss 1.029851
Epoch 9: Train Loss 1.013691 |  Val Loss 1.019029
Epoch 10: Train Loss 1.002872 |  Val Loss 1.008409
Epoch 11: Train Loss 0.992251 |  Val Loss 0.997990
Epoch 12: Train Loss 0.981829 |  Val Loss 0.987785
Epoch 13: Train Loss 0.971607 |  Val Loss 0.977770
Epoch 14: Train Loss 0.961590 |  Val Loss 0.967950
Epoch 15: Train Loss 0.951789 |  Val Loss 0.958332
Epoch 16: Train Loss 0.942203 |  Val Loss 0.948926
Epoch 17: Train Loss 0.932828 |  Val Los

[I 2025-01-04 14:59:48,250] Trial 0 finished with value: 0.6236095644623235 and parameters: {'n_trees': 8, 'n_selected': 12, 'n_layers': 1, 'mask_type': 'sparsemax'}. Best is trial 0 with value: 0.6236095644623235.


Epoch 792: Train Loss 0.410702 |  Val Loss 0.436318
Epoch 793: Train Loss 0.410442 |  Val Loss 0.436305
Epoch 794: Train Loss 0.410184 |  Val Loss 0.436314
Epoch 795: Train Loss 0.409922 |  Val Loss 0.436343
Epoch 796: Train Loss 0.409658 |  Val Loss 0.436386
Epoch 797: Train Loss 0.409387 |  Val Loss 0.436421
Epoch 798: Train Loss 0.409113 |  Val Loss 0.436459
Epoch 799: Train Loss 0.408843 |  Val Loss 0.436512
Epoch 800: Train Loss 0.408579 |  Val Loss 0.436544
Epoch 801: Train Loss 0.408319 |  Val Loss 0.436554
Epoch 802: Train Loss 0.408055 |  Val Loss 0.436535
Epoch 803: Train Loss 0.407795 |  Val Loss 0.436555
Epoch 804: Train Loss 0.407552 |  Val Loss 0.436563
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 1.186075 |  Val Loss 1.113365
Epoch 1: Train Loss 1.085647 |  Val Loss 1.021192
Epoch 2: Train Loss 1.000612 |  Val Loss 0.932381
Epoch 3: Train Loss 0.921692 |  Val Loss 0.854454
Epoch 4: Tra

[I 2025-01-04 14:59:50,338] Trial 1 finished with value: 0.8446530265013353 and parameters: {'n_trees': 40, 'n_selected': 24, 'n_layers': 7, 'mask_type': 'entmax'}. Best is trial 1 with value: 0.8446530265013353.


...

Epoch 71: Train Loss 0.076019 |  Val Loss 0.360772
Epoch 72: Train Loss 0.073563 |  Val Loss 0.368134
Epoch 73: Train Loss 0.071011 |  Val Loss 0.370187
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 1.491199 |  Val Loss 1.350551
Epoch 1: Train Loss 1.360421 |  Val Loss 1.234620
Epoch 2: Train Loss 1.245867 |  Val Loss 1.135391
Epoch 3: Train Loss 1.146863 |  Val Loss 1.047359
Epoch 4: Train Loss 1.058219 |  Val Loss 0.967321
Epoch 5: Train Loss 0.976354 |  Val Loss 0.894484
Epoch 6: Train Loss 0.903329 |  Val Loss 0.829042
Epoch 7: Train Loss 0.837128 |  Val Loss 0.768178
Epoch 8: Train Loss 0.777676 |  Val Loss 0.714271
Epoch 9: Train Loss 0.723010 |  Val Loss 0.664970
Epoch 10: Train Loss 0.672551 |  Val Loss 0.626083
Epoch 11: Train Loss 0.627098 |  Val Loss 0.590615
Epoch 12: Train Loss 0.586280 |  Val Loss 0.558881
Epoch 13: Train Loss 0.550225 |  Val Loss 0.532251
Epoch 14: Train Loss 0.518273 |

[I 2025-01-04 15:00:42,228] Trial 13 finished with value: 0.7785470932752259 and parameters: {'n_trees': 8, 'n_selected': 24, 'n_layers': 6, 'mask_type': 'entmax'}. Best is trial 4 with value: 0.8463272660560106.


Epoch 73: Train Loss 0.065501 |  Val Loss 0.286840
Epoch 74: Train Loss 0.062867 |  Val Loss 0.284859
Epoch 75: Train Loss 0.060459 |  Val Loss 0.281643
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.766249 |  Val Loss 0.729964
Epoch 1: Train Loss 0.719451 |  Val Loss 0.694520
Epoch 2: Train Loss 0.687354 |  Val Loss 0.665615
Epoch 3: Train Loss 0.660826 |  Val Loss 0.640157
Epoch 4: Train Loss 0.637474 |  Val Loss 0.619393
Epoch 5: Train Loss 0.617961 |  Val Loss 0.599113
Epoch 6: Train Loss 0.598996 |  Val Loss 0.583776
Epoch 7: Train Loss 0.583440 |  Val Loss 0.569893
Epoch 8: Train Loss 0.568926 |  Val Loss 0.555789
Epoch 9: Train Loss 0.555207 |  Val Loss 0.544740
Epoch 10: Train Loss 0.542076 |  Val Loss 0.535706
Epoch 11: Train Loss 0.529531 |  Val Loss 0.527839
Epoch 12: Train Loss 0.517277 |  Val Loss 0.519196
Epoch 13: Train Loss 0.504757 |  Val Loss 0.510177
Epoch 14: Train Loss 0.492422 |

[I 2025-01-04 15:00:45,222] Trial 14 finished with value: 0.8701109590936624 and parameters: {'n_trees': 80, 'n_selected': 16, 'n_layers': 7, 'mask_type': 'entmax'}. Best is trial 14 with value: 0.8701109590936624.


Epoch 84: Train Loss 0.054172 |  Val Loss 0.247670
Epoch 85: Train Loss 0.053250 |  Val Loss 0.238296
Epoch 86: Train Loss 0.052180 |  Val Loss 0.241385
Epoch 87: Train Loss 0.050788 |  Val Loss 0.238098
Epoch 88: Train Loss 0.049713 |  Val Loss 0.233745
Validation loss has not improved for 10 steps!
Early stopping applies.
Best hyperparameters: {'n_trees': 80, 'n_selected': 16, 'n_layers': 7, 'mask_type': 'entmax'}
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.766249 |  Val Loss 0.729964
Epoch 1: Train Loss 0.719451 |  Val Loss 0.694520
Epoch 2: Train Loss 0.687354 |  Val Loss 0.665615
Epoch 3: Train Loss 0.660826 |  Val Loss 0.640157
Epoch 4: Train Loss 0.637474 |  Val Loss 0.619393
Epoch 5: Train Loss 0.617961 |  Val Loss 0.599113
Epoch 6: Train Loss 0.598996 |  Val Loss 0.583776
Epoch 7: Train Loss 0.583440 |  Val Loss 0.569893
Epoch 8: Train Loss 0.568926 |  Val Loss 0.555789
Epoch 9: Train Loss 0.555207 |  Val Loss 0.544740
Epoch 10: Train Loss 0.542076 |  Val Lo

In [25]:
start_time = time.time()
II_450_pcc95_results_df = train_with_best_hyperparameters(datasets, 'II_450_pcc95')
print("--- %s seconds ---" % (time.time() - start_time))

[I 2025-01-04 15:01:08,171] A new study created in memory with name: no-name-b7dc3629-4eb2-4391-bb93-f7c2c2f78cb7


Currently training NCART model with dataset key: II_450_pcc95
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.734495 |  Val Loss 0.724973
Epoch 1: Train Loss 0.705659 |  Val Loss 0.696921
Epoch 2: Train Loss 0.683226 |  Val Loss 0.676063
Epoch 3: Train Loss 0.666376 |  Val Loss 0.660307
Epoch 4: Train Loss 0.654252 |  Val Loss 0.646992
Epoch 5: Train Loss 0.645800 |  Val Loss 0.637472
Epoch 6: Train Loss 0.638998 |  Val Loss 0.629927
Epoch 7: Train Loss 0.632684 |  Val Loss 0.620195
Epoch 8: Train Loss 0.626158 |  Val Loss 0.611064
Epoch 9: Train Loss 0.618875 |  Val Loss 0.603290
Epoch 10: Train Loss 0.611135 |  Val Loss 0.595890
Epoch 11: Train Loss 0.603541 |  Val Loss 0.587405
Epoch 12: Train Loss 0.595901 |  Val Loss 0.578605
Epoch 13: Train Loss 0.588157 |  Val Loss 0.569632
Epoch 14: Train Loss 0.580975 |  Val Loss 0.561565
Epoch 15: Train Loss 0.574647 |  Val Loss 0.554846
Epoch 16: Train Loss 0.568738 |  Val Loss 0.549106
Epoch 17: Train Loss 0.563152 |  Val Los

[I 2025-01-04 15:01:09,904] Trial 0 finished with value: 0.8222222222222222 and parameters: {'n_trees': 104, 'n_selected': 24, 'n_layers': 3, 'mask_type': 'sparsemax'}. Best is trial 0 with value: 0.8222222222222222.


Epoch 96: Train Loss 0.053685 |  Val Loss 0.242528
Epoch 97: Train Loss 0.052124 |  Val Loss 0.244433
Epoch 98: Train Loss 0.049993 |  Val Loss 0.255220
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.693156 |  Val Loss 0.691748
Epoch 1: Train Loss 0.692895 |  Val Loss 0.691494
Epoch 2: Train Loss 0.692644 |  Val Loss 0.691249
Epoch 3: Train Loss 0.692401 |  Val Loss 0.691011
Epoch 4: Train Loss 0.692168 |  Val Loss 0.690782
Epoch 5: Train Loss 0.691944 |  Val Loss 0.690562
Epoch 6: Train Loss 0.691728 |  Val Loss 0.690348
Epoch 7: Train Loss 0.691519 |  Val Loss 0.690139
Epoch 8: Train Loss 0.691318 |  Val Loss 0.689935
Epoch 9: Train Loss 0.691123 |  Val Loss 0.689734
Epoch 10: Train Loss 0.690932 |  Val Loss 0.689534
Epoch 11: Train Loss 0.690747 |  Val Loss 0.689333
Epoch 12: Train Loss 0.690566 |  Val Loss 0.689132
Epoch 13: Train Loss 0.690389 |  Val Loss 0.688930
Epoch 14: Train Loss 0.690214 |

[I 2025-01-04 15:01:24,135] Trial 1 finished with value: 0.6916266597923983 and parameters: {'n_trees': 72, 'n_selected': 4, 'n_layers': 1, 'mask_type': 'entmax'}. Best is trial 0 with value: 0.8222222222222222.


...

Epoch 83: Train Loss 0.048889 |  Val Loss 0.250759
Epoch 84: Train Loss 0.047781 |  Val Loss 0.251934
Epoch 85: Train Loss 0.046701 |  Val Loss 0.253667
Epoch 86: Train Loss 0.045667 |  Val Loss 0.256440
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.712505 |  Val Loss 0.696943
Epoch 1: Train Loss 0.689557 |  Val Loss 0.675797
Epoch 2: Train Loss 0.672381 |  Val Loss 0.660460
Epoch 3: Train Loss 0.658899 |  Val Loss 0.647833
Epoch 4: Train Loss 0.647305 |  Val Loss 0.636108
Epoch 5: Train Loss 0.637080 |  Val Loss 0.626547
Epoch 6: Train Loss 0.627604 |  Val Loss 0.616075
Epoch 7: Train Loss 0.618797 |  Val Loss 0.605819
Epoch 8: Train Loss 0.610103 |  Val Loss 0.595550
Epoch 9: Train Loss 0.601672 |  Val Loss 0.586061
Epoch 10: Train Loss 0.593087 |  Val Loss 0.577403
Epoch 11: Train Loss 0.584449 |  Val Loss 0.568285
Epoch 12: Train Loss 0.576054 |  Val Loss 0.559031
Epoch 13: Train Loss 0.567660 |

[I 2025-01-04 15:01:57,540] Trial 13 finished with value: 0.7847849263290312 and parameters: {'n_trees': 56, 'n_selected': 16, 'n_layers': 6, 'mask_type': 'entmax'}. Best is trial 4 with value: 0.8701109590936624.


Epoch 78: Train Loss 0.053719 |  Val Loss 0.269887
Epoch 79: Train Loss 0.051747 |  Val Loss 0.269532
Epoch 80: Train Loss 0.049895 |  Val Loss 0.278417
Epoch 81: Train Loss 0.048213 |  Val Loss 0.279309
Epoch 82: Train Loss 0.046542 |  Val Loss 0.273196
Epoch 83: Train Loss 0.044944 |  Val Loss 0.272025
Epoch 84: Train Loss 0.043490 |  Val Loss 0.275574
Epoch 85: Train Loss 0.042050 |  Val Loss 0.280214
Epoch 86: Train Loss 0.040734 |  Val Loss 0.272146
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.718698 |  Val Loss 0.664473
Epoch 1: Train Loss 0.668496 |  Val Loss 0.630794
Epoch 2: Train Loss 0.639911 |  Val Loss 0.610780
Epoch 3: Train Loss 0.622796 |  Val Loss 0.598556
Epoch 4: Train Loss 0.611476 |  Val Loss 0.587773
Epoch 5: Train Loss 0.600893 |  Val Loss 0.573748
Epoch 6: Train Loss 0.588088 |  Val Loss 0.557397
Epoch 7: Train Loss 0.573622 |  Val Loss 0.540632
Epoch 8: Train Loss 0.557846 

[I 2025-01-04 15:02:00,135] Trial 14 finished with value: 0.8446530265013353 and parameters: {'n_trees': 128, 'n_selected': 28, 'n_layers': 7, 'mask_type': 'entmax'}. Best is trial 4 with value: 0.8701109590936624.


Epoch 68: Train Loss 0.022547 |  Val Loss 0.226993
Epoch 69: Train Loss 0.022142 |  Val Loss 0.243350
Validation loss has not improved for 10 steps!
Early stopping applies.
Best hyperparameters: {'n_trees': 72, 'n_selected': 28, 'n_layers': 4, 'mask_type': 'sparsemax'}
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.785283 |  Val Loss 0.734660
Epoch 1: Train Loss 0.722480 |  Val Loss 0.686820
Epoch 2: Train Loss 0.674668 |  Val Loss 0.650502
Epoch 3: Train Loss 0.639266 |  Val Loss 0.624445
Epoch 4: Train Loss 0.614637 |  Val Loss 0.606209
Epoch 5: Train Loss 0.597976 |  Val Loss 0.593590
Epoch 6: Train Loss 0.587962 |  Val Loss 0.585280
Epoch 7: Train Loss 0.581618 |  Val Loss 0.579082
Epoch 8: Train Loss 0.576628 |  Val Loss 0.572575
Epoch 9: Train Loss 0.571072 |  Val Loss 0.565522
Epoch 10: Train Loss 0.564306 |  Val Loss 0.555671
Epoch 11: Train Loss 0.555143 |  Val Loss 0.544514
Epoch 12: Train Loss 0.544470 |  Val Loss 0.531991
Epoch 13: Train Loss 0.532343 |  Val

In [26]:
start_time = time.time()
II_450_pcc75_results_df = train_with_best_hyperparameters(datasets, 'II_450_pcc75')
print("--- %s seconds ---" % (time.time() - start_time))

[I 2025-01-04 15:02:15,044] A new study created in memory with name: no-name-75260ae9-c65c-4f1c-81ee-52596c56245b


Currently training NCART model with dataset key: II_450_pcc75
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.751066 |  Val Loss 0.742550
Epoch 1: Train Loss 0.732251 |  Val Loss 0.724575
Epoch 2: Train Loss 0.715403 |  Val Loss 0.709032
Epoch 3: Train Loss 0.700847 |  Val Loss 0.695690
Epoch 4: Train Loss 0.688359 |  Val Loss 0.683687
Epoch 5: Train Loss 0.677201 |  Val Loss 0.672537
Epoch 6: Train Loss 0.667325 |  Val Loss 0.662653
Epoch 7: Train Loss 0.658532 |  Val Loss 0.653902
Epoch 8: Train Loss 0.650886 |  Val Loss 0.646269
Epoch 9: Train Loss 0.644249 |  Val Loss 0.639598
Epoch 10: Train Loss 0.638593 |  Val Loss 0.633903
Epoch 11: Train Loss 0.633864 |  Val Loss 0.628922
Epoch 12: Train Loss 0.629694 |  Val Loss 0.624580
Epoch 13: Train Loss 0.626119 |  Val Loss 0.621059
Epoch 14: Train Loss 0.622880 |  Val Loss 0.617504
Epoch 15: Train Loss 0.619747 |  Val Loss 0.613622
Epoch 16: Train Loss 0.616605 |  Val Loss 0.609773
Epoch 17: Train Loss 0.613286 |  Val Los

[I 2025-01-04 15:02:17,961] Trial 0 finished with value: 0.8222222222222222 and parameters: {'n_trees': 128, 'n_selected': 16, 'n_layers': 4, 'mask_type': 'entmax'}. Best is trial 0 with value: 0.8222222222222222.


Epoch 123: Train Loss 0.070458 |  Val Loss 0.273302
Epoch 124: Train Loss 0.068871 |  Val Loss 0.277596
Epoch 125: Train Loss 0.067210 |  Val Loss 0.279219
Epoch 126: Train Loss 0.065624 |  Val Loss 0.273298
Epoch 127: Train Loss 0.064072 |  Val Loss 0.276974
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.795700 |  Val Loss 0.799803
Epoch 1: Train Loss 0.791328 |  Val Loss 0.795446
Epoch 2: Train Loss 0.787054 |  Val Loss 0.791185
Epoch 3: Train Loss 0.782880 |  Val Loss 0.787021
Epoch 4: Train Loss 0.778808 |  Val Loss 0.782961
Epoch 5: Train Loss 0.774843 |  Val Loss 0.779002
Epoch 6: Train Loss 0.770979 |  Val Loss 0.775146
Epoch 7: Train Loss 0.767219 |  Val Loss 0.771392
Epoch 8: Train Loss 0.763565 |  Val Loss 0.767743
Epoch 9: Train Loss 0.760014 |  Val Loss 0.764199
Epoch 10: Train Loss 0.756570 |  Val Loss 0.760762
Epoch 11: Train Loss 0.753235 |  Val Loss 0.757430
Epoch 12: Train Loss 0.750

[I 2025-01-04 15:02:31,820] Trial 1 finished with value: 0.5790660241435862 and parameters: {'n_trees': 24, 'n_selected': 8, 'n_layers': 1, 'mask_type': 'sparsemax'}. Best is trial 0 with value: 0.8222222222222222.


...

Epoch 127: Train Loss 0.047748 |  Val Loss 0.302201
Epoch 128: Train Loss 0.045485 |  Val Loss 0.297835
Epoch 129: Train Loss 0.043611 |  Val Loss 0.292733
Epoch 130: Train Loss 0.042611 |  Val Loss 0.305923
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.732288 |  Val Loss 0.700528
Epoch 1: Train Loss 0.707699 |  Val Loss 0.681826
Epoch 2: Train Loss 0.689530 |  Val Loss 0.669709
Epoch 3: Train Loss 0.677413 |  Val Loss 0.662143
Epoch 4: Train Loss 0.670306 |  Val Loss 0.658537
Epoch 5: Train Loss 0.666603 |  Val Loss 0.656222
Epoch 6: Train Loss 0.664203 |  Val Loss 0.654338
Epoch 7: Train Loss 0.661187 |  Val Loss 0.650214
Epoch 8: Train Loss 0.657133 |  Val Loss 0.644576
Epoch 9: Train Loss 0.652059 |  Val Loss 0.638304
Epoch 10: Train Loss 0.646197 |  Val Loss 0.631997
Epoch 11: Train Loss 0.640165 |  Val Loss 0.625926
Epoch 12: Train Loss 0.634456 |  Val Loss 0.620440
Epoch 13: Train Loss 0.6292

[I 2025-01-04 15:03:23,877] Trial 13 finished with value: 0.8001976040538966 and parameters: {'n_trees': 88, 'n_selected': 24, 'n_layers': 2, 'mask_type': 'entmax'}. Best is trial 7 with value: 0.8230354986052387.


Epoch 211: Train Loss 0.030300 |  Val Loss 0.217832
Epoch 212: Train Loss 0.029719 |  Val Loss 0.219309
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.750049 |  Val Loss 0.718549
Epoch 1: Train Loss 0.718496 |  Val Loss 0.692598
Epoch 2: Train Loss 0.692499 |  Val Loss 0.672090
Epoch 3: Train Loss 0.671795 |  Val Loss 0.656325
Epoch 4: Train Loss 0.656284 |  Val Loss 0.643686
Epoch 5: Train Loss 0.644288 |  Val Loss 0.634256
Epoch 6: Train Loss 0.635653 |  Val Loss 0.625994
Epoch 7: Train Loss 0.628213 |  Val Loss 0.617764
Epoch 8: Train Loss 0.621050 |  Val Loss 0.610007
Epoch 9: Train Loss 0.613699 |  Val Loss 0.602516
Epoch 10: Train Loss 0.606046 |  Val Loss 0.593450
Epoch 11: Train Loss 0.598030 |  Val Loss 0.583894
Epoch 12: Train Loss 0.589870 |  Val Loss 0.573980
Epoch 13: Train Loss 0.581336 |  Val Loss 0.564294
Epoch 14: Train Loss 0.572872 |  Val Loss 0.554491
Epoch 15: Train Loss 0.564462

[I 2025-01-04 15:03:26,153] Trial 14 finished with value: 0.7785470932752259 and parameters: {'n_trees': 128, 'n_selected': 24, 'n_layers': 4, 'mask_type': 'sparsemax'}. Best is trial 7 with value: 0.8230354986052387.


Epoch 86: Train Loss 0.051521 |  Val Loss 0.282789
Epoch 87: Train Loss 0.050006 |  Val Loss 0.274350
Epoch 88: Train Loss 0.047010 |  Val Loss 0.248985
Epoch 89: Train Loss 0.045655 |  Val Loss 0.258086
Epoch 90: Train Loss 0.043544 |  Val Loss 0.287002
Epoch 91: Train Loss 0.041887 |  Val Loss 0.286865
Epoch 92: Train Loss 0.040189 |  Val Loss 0.263666
Epoch 93: Train Loss 0.038487 |  Val Loss 0.262414
Epoch 94: Train Loss 0.036945 |  Val Loss 0.280742
Epoch 95: Train Loss 0.035710 |  Val Loss 0.282081
Validation loss has not improved for 10 steps!
Early stopping applies.
Best hyperparameters: {'n_trees': 40, 'n_selected': 32, 'n_layers': 3, 'mask_type': 'sparsemax'}
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.669446 |  Val Loss 0.653854
Epoch 1: Train Loss 0.639377 |  Val Loss 0.630344
Epoch 2: Train Loss 0.623827 |  Val Loss 0.614761
Epoch 3: Train Loss 0.613133 |  Val Loss 0.597116
Epoch 4: Train Loss 0.600583 |  Val Loss 0.579632
Epoch 5: Train Loss 0.587079 | 

In [27]:
start_time = time.time()
II_350_nopcc_results_df = train_with_best_hyperparameters(datasets, 'II_350_nopcc')
print("--- %s seconds ---" % (time.time() - start_time))

[I 2025-01-04 15:03:42,292] A new study created in memory with name: no-name-9270567d-d5bb-4492-b370-d9338bc73104


Currently training NCART model with dataset key: II_350_nopcc
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.891096 |  Val Loss 0.804806
Epoch 1: Train Loss 0.788572 |  Val Loss 0.724287
Epoch 2: Train Loss 0.707005 |  Val Loss 0.661878
Epoch 3: Train Loss 0.644543 |  Val Loss 0.619028
Epoch 4: Train Loss 0.598772 |  Val Loss 0.588199
Epoch 5: Train Loss 0.568031 |  Val Loss 0.565408
Epoch 6: Train Loss 0.548524 |  Val Loss 0.551845
Epoch 7: Train Loss 0.537254 |  Val Loss 0.542373
Epoch 8: Train Loss 0.529327 |  Val Loss 0.531806
Epoch 9: Train Loss 0.521917 |  Val Loss 0.521749
Epoch 10: Train Loss 0.513167 |  Val Loss 0.508998
Epoch 11: Train Loss 0.501657 |  Val Loss 0.494768
Epoch 12: Train Loss 0.486944 |  Val Loss 0.479245
Epoch 13: Train Loss 0.470588 |  Val Loss 0.461212
Epoch 14: Train Loss 0.453004 |  Val Loss 0.443699
Epoch 15: Train Loss 0.434888 |  Val Loss 0.427822
Epoch 16: Train Loss 0.417626 |  Val Loss 0.413848
Epoch 17: Train Loss 0.401806 |  Val Los

[I 2025-01-04 15:03:43,315] Trial 0 finished with value: 0.8003267306650412 and parameters: {'n_trees': 56, 'n_selected': 32, 'n_layers': 5, 'mask_type': 'entmax'}. Best is trial 0 with value: 0.8003267306650412.


Epoch 46: Train Loss 0.083321 |  Val Loss 0.301320
Epoch 47: Train Loss 0.076677 |  Val Loss 0.308434
Epoch 48: Train Loss 0.071438 |  Val Loss 0.299709
Epoch 49: Train Loss 0.065471 |  Val Loss 0.296211
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.699573 |  Val Loss 0.688636
Epoch 1: Train Loss 0.690412 |  Val Loss 0.679671
Epoch 2: Train Loss 0.681521 |  Val Loss 0.673648
Epoch 3: Train Loss 0.674786 |  Val Loss 0.668571
Epoch 4: Train Loss 0.668944 |  Val Loss 0.663718
Epoch 5: Train Loss 0.663452 |  Val Loss 0.659051
Epoch 6: Train Loss 0.658527 |  Val Loss 0.654286
Epoch 7: Train Loss 0.653832 |  Val Loss 0.650106
Epoch 8: Train Loss 0.649323 |  Val Loss 0.646100
Epoch 9: Train Loss 0.644915 |  Val Loss 0.642733
Epoch 10: Train Loss 0.640652 |  Val Loss 0.638954
Epoch 11: Train Loss 0.636420 |  Val Loss 0.634953
Epoch 12: Train Loss 0.632153 |  Val Loss 0.630765
Epoch 13: Train Loss 0.627875 |

[I 2025-01-04 15:03:46,270] Trial 1 finished with value: 0.8285714285714286 and parameters: {'n_trees': 120, 'n_selected': 12, 'n_layers': 3, 'mask_type': 'sparsemax'}. Best is trial 1 with value: 0.8285714285714286.


...

Epoch 122: Train Loss 0.054346 |  Val Loss 0.216400
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.945182 |  Val Loss 0.897307
Epoch 1: Train Loss 0.892065 |  Val Loss 0.853152
Epoch 2: Train Loss 0.847757 |  Val Loss 0.814431
Epoch 3: Train Loss 0.807740 |  Val Loss 0.777317
Epoch 4: Train Loss 0.770788 |  Val Loss 0.744059
Epoch 5: Train Loss 0.736799 |  Val Loss 0.713667
Epoch 6: Train Loss 0.705300 |  Val Loss 0.685898
Epoch 7: Train Loss 0.676539 |  Val Loss 0.661384
Epoch 8: Train Loss 0.650322 |  Val Loss 0.639876
Epoch 9: Train Loss 0.627028 |  Val Loss 0.619569
Epoch 10: Train Loss 0.606592 |  Val Loss 0.601511
Epoch 11: Train Loss 0.588548 |  Val Loss 0.585767
Epoch 12: Train Loss 0.572517 |  Val Loss 0.571743
Epoch 13: Train Loss 0.558098 |  Val Loss 0.558875
Epoch 14: Train Loss 0.544942 |  Val Loss 0.546689
Epoch 15: Train Loss 0.533450 |  Val Loss 0.535495
Epoch 16: Train Loss 0.523202 

[I 2025-01-04 15:04:35,614] Trial 13 finished with value: 0.8003267306650412 and parameters: {'n_trees': 72, 'n_selected': 20, 'n_layers': 5, 'mask_type': 'entmax'}. Best is trial 9 with value: 0.9146591207600471.


Epoch 85: Train Loss 0.057952 |  Val Loss 0.265979
Epoch 86: Train Loss 0.056318 |  Val Loss 0.264103
Epoch 87: Train Loss 0.054718 |  Val Loss 0.264716
Epoch 88: Train Loss 0.053141 |  Val Loss 0.268334
Epoch 89: Train Loss 0.051687 |  Val Loss 0.268685
Epoch 90: Train Loss 0.050287 |  Val Loss 0.266342
Epoch 91: Train Loss 0.048987 |  Val Loss 0.267036
Epoch 92: Train Loss 0.047678 |  Val Loss 0.268257
Epoch 93: Train Loss 0.046445 |  Val Loss 0.266894
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.781218 |  Val Loss 0.741809
Epoch 1: Train Loss 0.754228 |  Val Loss 0.721969
Epoch 2: Train Loss 0.732099 |  Val Loss 0.705147
Epoch 3: Train Loss 0.713473 |  Val Loss 0.689531
Epoch 4: Train Loss 0.695743 |  Val Loss 0.674359
Epoch 5: Train Loss 0.679387 |  Val Loss 0.660846
Epoch 6: Train Loss 0.664920 |  Val Loss 0.648487
Epoch 7: Train Loss 0.652208 |  Val Loss 0.637526
Epoch 8: Train Loss 0.640463 

[I 2025-01-04 15:04:38,588] Trial 14 finished with value: 0.8003267306650412 and parameters: {'n_trees': 56, 'n_selected': 12, 'n_layers': 4, 'mask_type': 'sparsemax'}. Best is trial 9 with value: 0.9146591207600471.


Epoch 142: Train Loss 0.052632 |  Val Loss 0.280845
Epoch 143: Train Loss 0.051808 |  Val Loss 0.278312
Epoch 144: Train Loss 0.050980 |  Val Loss 0.278076
Epoch 145: Train Loss 0.050207 |  Val Loss 0.280412
Epoch 146: Train Loss 0.049421 |  Val Loss 0.281836
Validation loss has not improved for 10 steps!
Early stopping applies.
Best hyperparameters: {'n_trees': 80, 'n_selected': 12, 'n_layers': 5, 'mask_type': 'entmax'}
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.737445 |  Val Loss 0.713592
Epoch 1: Train Loss 0.711074 |  Val Loss 0.693716
Epoch 2: Train Loss 0.689837 |  Val Loss 0.675091
Epoch 3: Train Loss 0.670718 |  Val Loss 0.661133
Epoch 4: Train Loss 0.655320 |  Val Loss 0.649962
Epoch 5: Train Loss 0.641586 |  Val Loss 0.638680
Epoch 6: Train Loss 0.628771 |  Val Loss 0.628764
Epoch 7: Train Loss 0.617482 |  Val Loss 0.619649
Epoch 8: Train Loss 0.607738 |  Val Loss 0.610639
Epoch 9: Train Loss 0.598438 |  Val Loss 0.602658
Epoch 10: Train Loss 0.589926 |  V

In [28]:
start_time = time.time()
II_350_pcc95_results_df = train_with_best_hyperparameters(datasets, 'II_350_pcc95')
print("--- %s seconds ---" % (time.time() - start_time))

[I 2025-01-04 15:05:02,144] A new study created in memory with name: no-name-6a4faf68-a730-432a-b2c8-edccf95447de


Currently training NCART model with dataset key: II_350_pcc95
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.843328 |  Val Loss 0.810370
Epoch 1: Train Loss 0.812769 |  Val Loss 0.786180
Epoch 2: Train Loss 0.786805 |  Val Loss 0.764362
Epoch 3: Train Loss 0.763864 |  Val Loss 0.745223
Epoch 4: Train Loss 0.743300 |  Val Loss 0.727337
Epoch 5: Train Loss 0.724630 |  Val Loss 0.710315
Epoch 6: Train Loss 0.707606 |  Val Loss 0.694103
Epoch 7: Train Loss 0.691439 |  Val Loss 0.679602
Epoch 8: Train Loss 0.676386 |  Val Loss 0.665994
Epoch 9: Train Loss 0.662146 |  Val Loss 0.652825
Epoch 10: Train Loss 0.648912 |  Val Loss 0.640528
Epoch 11: Train Loss 0.636510 |  Val Loss 0.629571
Epoch 12: Train Loss 0.625037 |  Val Loss 0.619586
Epoch 13: Train Loss 0.614330 |  Val Loss 0.609572
Epoch 14: Train Loss 0.604221 |  Val Loss 0.600159
Epoch 15: Train Loss 0.594403 |  Val Loss 0.591125
Epoch 16: Train Loss 0.585029 |  Val Loss 0.582703
Epoch 17: Train Loss 0.576015 |  Val Los

[I 2025-01-04 15:05:06,610] Trial 0 finished with value: 0.8871639008998208 and parameters: {'n_trees': 56, 'n_selected': 12, 'n_layers': 6, 'mask_type': 'entmax'}. Best is trial 0 with value: 0.8871639008998208.


On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.783969 |  Val Loss 0.740990
Epoch 1: Train Loss 0.745614 |  Val Loss 0.707565
Epoch 2: Train Loss 0.714079 |  Val Loss 0.680716
Epoch 3: Train Loss 0.688565 |  Val Loss 0.656798
Epoch 4: Train Loss 0.665817 |  Val Loss 0.637657
Epoch 5: Train Loss 0.645506 |  Val Loss 0.621295
Epoch 6: Train Loss 0.628144 |  Val Loss 0.606877
Epoch 7: Train Loss 0.612943 |  Val Loss 0.594128
Epoch 8: Train Loss 0.598867 |  Val Loss 0.583436
Epoch 9: Train Loss 0.585839 |  Val Loss 0.572738
Epoch 10: Train Loss 0.574234 |  Val Loss 0.561811
Epoch 11: Train Loss 0.563917 |  Val Loss 0.551268
Epoch 12: Train Loss 0.554709 |  Val Loss 0.541884
Epoch 13: Train Loss 0.546254 |  Val Loss 0.532735
Epoch 14: Train Loss 0.538288 |  Val Loss 0.524937
Epoch 15: Train Loss 0.530740 |  Val Loss 0.517865
Epoch 16: Train Loss 0.523404 |  Val Loss 0.511311
Epoch 17: Train Loss 0.515991 |  Val Loss 0.504838
Epoch 18: Train Loss 0.508541 |  Val Loss 0.498240


[I 2025-01-04 15:05:08,730] Trial 1 finished with value: 0.8003267306650412 and parameters: {'n_trees': 40, 'n_selected': 16, 'n_layers': 5, 'mask_type': 'entmax'}. Best is trial 0 with value: 0.8871639008998208.


...

Epoch 120: Train Loss 0.105750 |  Val Loss 0.341837
Epoch 121: Train Loss 0.102919 |  Val Loss 0.344161
Epoch 122: Train Loss 0.100034 |  Val Loss 0.346579
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.831628 |  Val Loss 0.815192
Epoch 1: Train Loss 0.816209 |  Val Loss 0.801896
Epoch 2: Train Loss 0.802777 |  Val Loss 0.789843
Epoch 3: Train Loss 0.790310 |  Val Loss 0.777945
Epoch 4: Train Loss 0.778923 |  Val Loss 0.767977
Epoch 5: Train Loss 0.768655 |  Val Loss 0.758956
Epoch 6: Train Loss 0.758643 |  Val Loss 0.749551
Epoch 7: Train Loss 0.749036 |  Val Loss 0.741426
Epoch 8: Train Loss 0.740237 |  Val Loss 0.733877
Epoch 9: Train Loss 0.732101 |  Val Loss 0.726007
Epoch 10: Train Loss 0.724408 |  Val Loss 0.717963
Epoch 11: Train Loss 0.717086 |  Val Loss 0.710681
Epoch 12: Train Loss 0.709969 |  Val Loss 0.703946
Epoch 13: Train Loss 0.703019 |  Val Loss 0.697729
Epoch 14: Train Loss 0.69639

[I 2025-01-04 15:06:04,442] Trial 13 finished with value: 0.8285714285714286 and parameters: {'n_trees': 24, 'n_selected': 8, 'n_layers': 4, 'mask_type': 'entmax'}. Best is trial 11 with value: 0.9146591207600471.


Epoch 278: Train Loss 0.033379 |  Val Loss 0.227676
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.843328 |  Val Loss 0.810370
Epoch 1: Train Loss 0.812769 |  Val Loss 0.786180
Epoch 2: Train Loss 0.786805 |  Val Loss 0.764362
Epoch 3: Train Loss 0.763864 |  Val Loss 0.745223
Epoch 4: Train Loss 0.743300 |  Val Loss 0.727337
Epoch 5: Train Loss 0.724630 |  Val Loss 0.710315
Epoch 6: Train Loss 0.707606 |  Val Loss 0.694103
Epoch 7: Train Loss 0.691439 |  Val Loss 0.679602
Epoch 8: Train Loss 0.676386 |  Val Loss 0.665994
Epoch 9: Train Loss 0.662146 |  Val Loss 0.652825
Epoch 10: Train Loss 0.648912 |  Val Loss 0.640528
Epoch 11: Train Loss 0.636510 |  Val Loss 0.629571
Epoch 12: Train Loss 0.625037 |  Val Loss 0.619586
Epoch 13: Train Loss 0.614330 |  Val Loss 0.609572
Epoch 14: Train Loss 0.604221 |  Val Loss 0.600159
Epoch 15: Train Loss 0.594403 |  Val Loss 0.591125
Epoch 16: Train Loss 0.585029 

[I 2025-01-04 15:06:09,174] Trial 14 finished with value: 0.8871639008998208 and parameters: {'n_trees': 56, 'n_selected': 12, 'n_layers': 6, 'mask_type': 'entmax'}. Best is trial 11 with value: 0.9146591207600471.


Epoch 200: Train Loss 0.023000 |  Val Loss 0.161965
Epoch 201: Train Loss 0.022749 |  Val Loss 0.161498
Epoch 202: Train Loss 0.022503 |  Val Loss 0.161299
Epoch 203: Train Loss 0.022261 |  Val Loss 0.162278
Epoch 204: Train Loss 0.022023 |  Val Loss 0.162415
Epoch 205: Train Loss 0.021789 |  Val Loss 0.161556
Epoch 206: Train Loss 0.021559 |  Val Loss 0.161856
Validation loss has not improved for 10 steps!
Early stopping applies.
Best hyperparameters: {'n_trees': 24, 'n_selected': 12, 'n_layers': 6, 'mask_type': 'entmax'}
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.691009 |  Val Loss 0.678430
Epoch 1: Train Loss 0.674212 |  Val Loss 0.669365
Epoch 2: Train Loss 0.661092 |  Val Loss 0.658918
Epoch 3: Train Loss 0.649707 |  Val Loss 0.647881
Epoch 4: Train Loss 0.639488 |  Val Loss 0.639885
Epoch 5: Train Loss 0.630509 |  Val Loss 0.633061
Epoch 6: Train Loss 0.622125 |  Val Loss 0.626011
Epoch 7: Train Loss 0.614215 |  Val Loss 0.618509
Epoch 8: Train Loss 0.606438 |

In [29]:
start_time = time.time()
II_350_pcc75_results_df = train_with_best_hyperparameters(datasets, 'II_350_pcc75')
print("--- %s seconds ---" % (time.time() - start_time))

[I 2025-01-04 15:06:33,398] A new study created in memory with name: no-name-17805b0f-91bf-4d20-87d4-a1d408dcc44b


Currently training NCART model with dataset key: II_350_pcc75
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 1.199563 |  Val Loss 1.106670
Epoch 1: Train Loss 1.090424 |  Val Loss 1.007220
Epoch 2: Train Loss 0.992527 |  Val Loss 0.919733
Epoch 3: Train Loss 0.905738 |  Val Loss 0.843055
Epoch 4: Train Loss 0.828314 |  Val Loss 0.776596
Epoch 5: Train Loss 0.762591 |  Val Loss 0.721484
Epoch 6: Train Loss 0.708766 |  Val Loss 0.679053
Epoch 7: Train Loss 0.667447 |  Val Loss 0.648402
Epoch 8: Train Loss 0.637010 |  Val Loss 0.627922
Epoch 9: Train Loss 0.617038 |  Val Loss 0.617073
Epoch 10: Train Loss 0.605945 |  Val Loss 0.612968
Epoch 11: Train Loss 0.601566 |  Val Loss 0.612870
Epoch 12: Train Loss 0.601026 |  Val Loss 0.613861
Epoch 13: Train Loss 0.601832 |  Val Loss 0.613585
Epoch 14: Train Loss 0.603261 |  Val Loss 0.611888
Epoch 15: Train Loss 0.600741 |  Val Loss 0.607291
Epoch 16: Train Loss 0.596450 |  Val Loss 0.600199
Epoch 17: Train Loss 0.588774 |  Val Los

[I 2025-01-04 15:06:35,035] Trial 0 finished with value: 0.8299275201966065 and parameters: {'n_trees': 64, 'n_selected': 32, 'n_layers': 6, 'mask_type': 'entmax'}. Best is trial 0 with value: 0.8299275201966065.


Epoch 74: Train Loss 0.044494 |  Val Loss 0.226043
Epoch 75: Train Loss 0.042487 |  Val Loss 0.226501
Epoch 76: Train Loss 0.040678 |  Val Loss 0.224881
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.734381 |  Val Loss 0.723653
Epoch 1: Train Loss 0.727050 |  Val Loss 0.717164
Epoch 2: Train Loss 0.720373 |  Val Loss 0.711178
Epoch 3: Train Loss 0.714307 |  Val Loss 0.705387
Epoch 4: Train Loss 0.708220 |  Val Loss 0.699842
Epoch 5: Train Loss 0.702553 |  Val Loss 0.694504
Epoch 6: Train Loss 0.697248 |  Val Loss 0.689609
Epoch 7: Train Loss 0.692309 |  Val Loss 0.685219
Epoch 8: Train Loss 0.687644 |  Val Loss 0.681480
Epoch 9: Train Loss 0.683271 |  Val Loss 0.678532
Epoch 10: Train Loss 0.679271 |  Val Loss 0.675192
Epoch 11: Train Loss 0.675441 |  Val Loss 0.671099
Epoch 12: Train Loss 0.671756 |  Val Loss 0.667616
Epoch 13: Train Loss 0.668085 |  Val Loss 0.664200
Epoch 14: Train Loss 0.664101 |

[I 2025-01-04 15:06:41,140] Trial 1 finished with value: 0.8285714285714286 and parameters: {'n_trees': 16, 'n_selected': 8, 'n_layers': 7, 'mask_type': 'sparsemax'}. Best is trial 0 with value: 0.8299275201966065.


...

Epoch 97: Train Loss 0.053069 |  Val Loss 0.161636
Epoch 98: Train Loss 0.050390 |  Val Loss 0.163442
Epoch 99: Train Loss 0.047977 |  Val Loss 0.165494
Epoch 100: Train Loss 0.045686 |  Val Loss 0.167324
Epoch 101: Train Loss 0.043604 |  Val Loss 0.171623
Epoch 102: Train Loss 0.041619 |  Val Loss 0.173210
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.739846 |  Val Loss 0.691132
Epoch 1: Train Loss 0.697508 |  Val Loss 0.659104
Epoch 2: Train Loss 0.665241 |  Val Loss 0.636836
Epoch 3: Train Loss 0.645135 |  Val Loss 0.625709
Epoch 4: Train Loss 0.633215 |  Val Loss 0.619242
Epoch 5: Train Loss 0.626602 |  Val Loss 0.615266
Epoch 6: Train Loss 0.622621 |  Val Loss 0.611573
Epoch 7: Train Loss 0.618197 |  Val Loss 0.606701
Epoch 8: Train Loss 0.612355 |  Val Loss 0.599696
Epoch 9: Train Loss 0.604707 |  Val Loss 0.591273
Epoch 10: Train Loss 0.595704 |  Val Loss 0.582268
Epoch 11: Train Loss 0.58637

[I 2025-01-04 15:07:26,092] Trial 13 finished with value: 0.7714285714285715 and parameters: {'n_trees': 56, 'n_selected': 28, 'n_layers': 3, 'mask_type': 'entmax'}. Best is trial 6 with value: 0.9428571428571428.


Epoch 96: Train Loss 0.098501 |  Val Loss 0.251772
Epoch 97: Train Loss 0.094247 |  Val Loss 0.259410
Epoch 98: Train Loss 0.090865 |  Val Loss 0.258165
Epoch 99: Train Loss 0.087191 |  Val Loss 0.253299
Epoch 100: Train Loss 0.084000 |  Val Loss 0.256109
Epoch 101: Train Loss 0.080614 |  Val Loss 0.262964
Epoch 102: Train Loss 0.077596 |  Val Loss 0.262543
Validation loss has not improved for 10 steps!
Early stopping applies.
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 0.751511 |  Val Loss 0.726559
Epoch 1: Train Loss 0.717813 |  Val Loss 0.707245
Epoch 2: Train Loss 0.697975 |  Val Loss 0.699756
Epoch 3: Train Loss 0.689906 |  Val Loss 0.698796
Epoch 4: Train Loss 0.688429 |  Val Loss 0.698430
Epoch 5: Train Loss 0.687683 |  Val Loss 0.695251
Epoch 6: Train Loss 0.684287 |  Val Loss 0.688829
Epoch 7: Train Loss 0.677827 |  Val Loss 0.680401
Epoch 8: Train Loss 0.669473 |  Val Loss 0.671550
Epoch 9: Train Loss 0.660787 |  Val Loss 0.663598
Epoch 10: Train Loss 0.65304

[I 2025-01-04 15:07:38,714] Trial 14 finished with value: 0.8857142857142857 and parameters: {'n_trees': 80, 'n_selected': 32, 'n_layers': 1, 'mask_type': 'sparsemax'}. Best is trial 6 with value: 0.9428571428571428.


Epoch 986: Train Loss 0.061562 |  Val Loss 0.169218
Epoch 987: Train Loss 0.061355 |  Val Loss 0.169145
Epoch 988: Train Loss 0.061148 |  Val Loss 0.169104
Epoch 989: Train Loss 0.060942 |  Val Loss 0.169037
Epoch 990: Train Loss 0.060738 |  Val Loss 0.168960
Epoch 991: Train Loss 0.060534 |  Val Loss 0.168927
Epoch 992: Train Loss 0.060330 |  Val Loss 0.168917
Epoch 993: Train Loss 0.060127 |  Val Loss 0.168894
Epoch 994: Train Loss 0.059925 |  Val Loss 0.168821
Epoch 995: Train Loss 0.059723 |  Val Loss 0.168703
Epoch 996: Train Loss 0.059521 |  Val Loss 0.168641
Epoch 997: Train Loss 0.059321 |  Val Loss 0.168629
Epoch 998: Train Loss 0.059122 |  Val Loss 0.168637
Epoch 999: Train Loss 0.058923 |  Val Loss 0.168585
Best hyperparameters: {'n_trees': 56, 'n_selected': 32, 'n_layers': 4, 'mask_type': 'sparsemax'}
On Device:  cuda:0
Binary classification
Epoch 0: Train Loss 1.039393 |  Val Loss 0.928782
Epoch 1: Train Loss 0.923544 |  Val Loss 0.821964
Epoch 2: Train Loss 0.819601 |  Va

In [ ]:
print("Tuned Model:")
print('\n---------------------------------------------------------------------\n')
view_results_df(II_all_nopcc_results_df, 'II_all_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_all_pcc95_results_df, 'II_all_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_all_nopcc_results_df, 'II_all_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_550_nopcc_results_df, 'II_550_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_550_pcc95_results_df, 'II_550_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_550_nopcc_results_df, 'II_550_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_450_nopcc_results_df, 'II_450_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_450_pcc95_results_df, 'II_450_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_450_nopcc_results_df, 'II_450_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_350_nopcc_results_df, 'II_350_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_350_pcc95_results_df, 'II_350_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(II_350_nopcc_results_df, 'II_350_pcc75')
print('\n---------------------------------------------------------------------\n')
